In [1]:
#Prints **all** console output, not just last item in cell 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

**Notebook author:** emeinhardt@ucsd.edu

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span><ul class="toc-item"><li><span><a href="#Requirements" data-toc-modified-id="Requirements-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Requirements</a></span></li><li><span><a href="#Usage" data-toc-modified-id="Usage-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Usage</a></span></li></ul></li><li><span><a href="#Parameters" data-toc-modified-id="Parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Imports-/-load-data" data-toc-modified-id="Imports-/-load-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Imports / load data</a></span><ul class="toc-item"><li><span><a href="#Load/extract-sanity-checking-data" data-toc-modified-id="Load/extract-sanity-checking-data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Load/extract sanity-checking data</a></span><ul class="toc-item"><li><span><a href="#Segmental-lexicon,-prefixes,-inventory,-and-triphones" data-toc-modified-id="Segmental-lexicon,-prefixes,-inventory,-and-triphones-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Segmental lexicon, prefixes, inventory, and triphones</a></span></li><li><span><a href="#Triphone-channel-distribution-and-channel-alphabet" data-toc-modified-id="Triphone-channel-distribution-and-channel-alphabet-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Triphone channel distribution and channel alphabet</a></span></li><li><span><a href="#Preview-and-postview-distributions-and-alphabets" data-toc-modified-id="Preview-and-postview-distributions-and-alphabets-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Preview and postview distributions and alphabets</a></span></li><li><span><a href="#Corpus-contexts" data-toc-modified-id="Corpus-contexts-3.1.4"><span class="toc-item-num">3.1.4&nbsp;&nbsp;</span>Corpus contexts</a></span></li><li><span><a href="#Conversion-to-one-hot-vectors-/-sequences-thereof" data-toc-modified-id="Conversion-to-one-hot-vectors-/-sequences-thereof-3.1.5"><span class="toc-item-num">3.1.5&nbsp;&nbsp;</span>Conversion to one-hot vectors / sequences thereof</a></span></li></ul></li><li><span><a href="#Load-segmental-sequence-channel-matrices-$p(Y_0^k|X_0^k)$-and-$p(Y_0^f|X_0^f)$" data-toc-modified-id="Load-segmental-sequence-channel-matrices-$p(Y_0^k|X_0^k)$-and-$p(Y_0^f|X_0^f)$-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Load segmental sequence channel matrices $p(Y_0^k|X_0^k)$ and $p(Y_0^f|X_0^f)$</a></span></li><li><span><a href="#Load-contextual-distribution-on-segmental-wordforms-$p(W|C)$" data-toc-modified-id="Load-contextual-distribution-on-segmental-wordforms-$p(W|C)$-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Load contextual distribution on segmental wordforms $p(W|C)$</a></span></li><li><span><a href="#Load-lexicon-metadata" data-toc-modified-id="Load-lexicon-metadata-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Load lexicon metadata</a></span></li></ul></li><li><span><a href="#Slow,-but-sanity-checking-calculations" data-toc-modified-id="Slow,-but-sanity-checking-calculations-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Slow, but sanity-checking calculations</a></span><ul class="toc-item"><li><span><a href="#$p(C_0^i-|-X_0^i;-X_{i+1})$" data-toc-modified-id="$p(C_0^i-|-X_0^i;-X_{i+1})$-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>$p(C_0^i | X_0^i; X_{i+1})$</a></span></li><li><span><a href="#$p(Y_0^i|X_0^k)$" data-toc-modified-id="$p(Y_0^i|X_0^k)$-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>$p(Y_0^i|X_0^k)$</a></span></li><li><span><a href="#$p(X_0^f|C)$" data-toc-modified-id="$p(X_0^f|C)$-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>$p(X_0^f|C)$</a></span></li><li><span><a href="#$p(Y_0^i-|-c)$" data-toc-modified-id="$p(Y_0^i-|-c)$-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>$p(Y_0^i | c)$</a></span></li><li><span><a href="#$p(Y_0^i|x_0^k)$-sampler" data-toc-modified-id="$p(Y_0^i|x_0^k)$-sampler-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>$p(Y_0^i|x_0^k)$ sampler</a></span></li><li><span><a href="#$p(\widehat{X_0^f}|Y_0^{i},-C)$" data-toc-modified-id="$p(\widehat{X_0^f}|Y_0^{i},-C)$-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>$p(\widehat{X_0^f}|Y_0^{i}, C)$</a></span></li><li><span><a href="#$\widehat{p}(\widehat{X_0^f}|X_0^i;X_{i+1},-C)$" data-toc-modified-id="$\widehat{p}(\widehat{X_0^f}|X_0^i;X_{i+1},-C)$-4.7"><span class="toc-item-num">4.7&nbsp;&nbsp;</span>$\widehat{p}(\widehat{X_0^f}|X_0^i;X_{i+1}, C)$</a></span></li></ul></li><li><span><a href="#Definition-of-core-calculation" data-toc-modified-id="Definition-of-core-calculation-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Definition of core calculation</a></span><ul class="toc-item"><li><span><a href="#Channel-sampler" data-toc-modified-id="Channel-sampler-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Channel sampler</a></span></li><li><span><a href="#Sample-parallel-posterior-estimator" data-toc-modified-id="Sample-parallel-posterior-estimator-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Sample-parallel posterior estimator</a></span></li><li><span><a href="#Testing" data-toc-modified-id="Testing-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Testing</a></span></li><li><span><a href="#Exact/full-wordform-length-variant-functions" data-toc-modified-id="Exact/full-wordform-length-variant-functions-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Exact/full wordform-length variant functions</a></span></li><li><span><a href="#Calculation-volume-organization-for-Case-1" data-toc-modified-id="Calculation-volume-organization-for-Case-1-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Calculation volume organization for Case 1</a></span><ul class="toc-item"><li><span><a href="#Slices:-Calculate-results-for-groups-of-reconstructed-wordforms-by-(input-prefix,-context)" data-toc-modified-id="Slices:-Calculate-results-for-groups-of-reconstructed-wordforms-by-(input-prefix,-context)-5.5.1"><span class="toc-item-num">5.5.1&nbsp;&nbsp;</span>Slices: Calculate results for groups of reconstructed wordforms by (input prefix, context)</a></span></li><li><span><a href="#Blocks:-Calculate-results-for-groups-of-input-prefixes-by-context" data-toc-modified-id="Blocks:-Calculate-results-for-groups-of-input-prefixes-by-context-5.5.2"><span class="toc-item-num">5.5.2&nbsp;&nbsp;</span>Blocks: Calculate results for groups of input prefixes by context</a></span></li></ul></li><li><span><a href="#Calculation-volume-organization-2" data-toc-modified-id="Calculation-volume-organization-2-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>Calculation volume organization 2</a></span></li><li><span><a href="#Calculation-volume-organization-3" data-toc-modified-id="Calculation-volume-organization-3-5.7"><span class="toc-item-num">5.7&nbsp;&nbsp;</span>Calculation volume organization 3</a></span><ul class="toc-item"><li><span><a href="#Slices:-Calculate-$\{p(\widehat{W}-=-w^*-|-W-=-w^*,-C-=-c)\}$-for-all-$W$-and-for-a-single-given-$c$" data-toc-modified-id="Slices:-Calculate-$\{p(\widehat{W}-=-w^*-|-W-=-w^*,-C-=-c)\}$-for-all-$W$-and-for-a-single-given-$c$-5.7.1"><span class="toc-item-num">5.7.1&nbsp;&nbsp;</span>Slices: Calculate $\{p(\widehat{W} = w^* | W = w^*, C = c)\}$ for all $W$ and for a single given $c$</a></span></li></ul></li></ul></li><li><span><a href="#Calculate-distribution" data-toc-modified-id="Calculate-distribution-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Calculate distribution</a></span><ul class="toc-item"><li><span><a href="#Number-of-calculations-necessary-under-different-calculation-criteria" data-toc-modified-id="Number-of-calculations-necessary-under-different-calculation-criteria-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Number of calculations necessary under different calculation criteria</a></span></li><li><span><a href="#Determine-dimensions-and-cells-to-be-calculated-vs.-ignored" data-toc-modified-id="Determine-dimensions-and-cells-to-be-calculated-vs.-ignored-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Determine dimensions and cells to be calculated vs. ignored</a></span><ul class="toc-item"><li><span><a href="#Option:-Only-calculate-full-source-wordforms-rather-than-all-source-prefixes?" data-toc-modified-id="Option:-Only-calculate-full-source-wordforms-rather-than-all-source-prefixes?-6.2.1"><span class="toc-item-num">6.2.1&nbsp;&nbsp;</span>Option: Only calculate full source wordforms rather than all source prefixes?</a></span></li><li><span><a href="#Suboption:-Given-$\neg-r$-and-a-full-source-wordform-$w^*$,-only-calculate-posterior-probability-$p(\widehat{W}-=-w^*-|-W-=-w^*,-C-=-c)$,-$\forall-w^*$?" data-toc-modified-id="Suboption:-Given-$\neg-r$-and-a-full-source-wordform-$w^*$,-only-calculate-posterior-probability-$p(\widehat{W}-=-w^*-|-W-=-w^*,-C-=-c)$,-$\forall-w^*$?-6.2.2"><span class="toc-item-num">6.2.2&nbsp;&nbsp;</span>Suboption: Given $\neg r$ and a full source wordform $w^*$, only calculate posterior probability $p(\widehat{W} = w^* | W = w^*, C = c)$, $\forall w^*$?</a></span></li><li><span><a href="#Suboption:-Restrict-calculations-only-to-(source-sequence,-reconstruction-sequence)-pairs-within-edit-distance-$k$?" data-toc-modified-id="Suboption:-Restrict-calculations-only-to-(source-sequence,-reconstruction-sequence)-pairs-within-edit-distance-$k$?-6.2.3"><span class="toc-item-num">6.2.3&nbsp;&nbsp;</span>Suboption: Restrict calculations only to (source sequence, reconstruction sequence) pairs within edit distance $k$?</a></span></li><li><span><a href="#Altogether..." data-toc-modified-id="Altogether...-6.2.4"><span class="toc-item-num">6.2.4&nbsp;&nbsp;</span>Altogether...</a></span></li></ul></li><li><span><a href="#Case-1:-$r-\land-(0-<-k-\leq-4)$" data-toc-modified-id="Case-1:-$r-\land-(0-<-k-\leq-4)$-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Case 1: $r \land (0 &lt; k \leq 4)$</a></span></li><li><span><a href="#Case-2:-$\neg-r-\land-(0-<-k-\leq-4)-\land-\neg-e$" data-toc-modified-id="Case-2:-$\neg-r-\land-(0-<-k-\leq-4)-\land-\neg-e$-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Case 2: $\neg r \land (0 &lt; k \leq 4) \land \neg e$</a></span></li><li><span><a href="#Case-3:-$\neg-r-\land-e$" data-toc-modified-id="Case-3:-$\neg-r-\land-e$-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>Case 3: $\neg r \land e$</a></span></li></ul></li></ul></div>

# Overview

Given a choice of parameters $\epsilon$ and $n$, and given
 - wordform channel matrices $p(Y_0^f|X_0^f)$
 - a contextual distribution on segmental wordforms $p(X_0^f|C)$
 - segmental lexicon metadata pre-calculating $k$-cousins/$k$-spheres up to $k=4$
 
Calculate

$$\hat{p}(\hat{X}_0^f = x_0^{'f}|X_0^f = x_0^{*f}, c) = \frac{1}{n} \sum\limits_{y_0^f \in S} p(\hat{X}_0^f = x_0^{'f}|y_0^f, c)$$
 where 
  - edit distance $d(x_0^{'f}, x_0^{*f}) \leq 4$
  - $S = $ a set of $n$ samples from $p(Y_0^f|x_0^{*f})$. In practice an $n \approx 50$ seems to result in estimates that are within $10^{-6}$ of the true estimate. 
  - $p(\hat{X}_0^f = x_0^{'f}|Y_0^f = y_0^f, c) = \frac{p(y_0^f|x_0^{'f})p(x_0^{'f}|c)}{p(y_0^f | c)}$
  - $p(y_0^f| c) = \sum\limits_{v', x_0^{''f}} p(y_0^f|x_0^{''f})p(x_0^{''f}|v')p(v'|c) = \sum\limits_{x_0^{''f}} p(y_0^f|x_0^{''f})p(x_0^{''f}|c)$

## Requirements

#FIXME

## Usage

#FIXME

# Parameters

In [2]:
from os import getcwd, chdir, listdir, path, mkdir, makedirs

In [3]:
from boilerplate import *

In [4]:
repo_dir = getcwd(); repo_dir

'/mnt/cube/home/AD/emeinhar/wr'

In [5]:
# Parameters

#p(Y_0^k|X_0^k) - from step 4e
c = ''
# c = 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/LTR_Buckeye_aligned_CM_filtered_LM_filtered_CMs_by_length_by_prefix_index.pickle'
c = 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_CMs_by_length_by_wordform_index.pickle'

#p(Y_0^f|X_0^f) - from step 4e
f = ''
f = 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_exact_CMs_by_length_by_wordform_index.pickle'

#p(X_0^f|C) - from step 4c
w = ''
# w = 'LD_Fisher_vocab_in_Buckeye_contexts/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_buckeye_contexts.pW_C.npy'
w = 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_C.npy'

# LTR metadata directory - contains pre-computed k-cousin/k-sphere information
m = ''
# m = 'LTR_Buckeye_aligned_w_GD_AmE_destressed'
m = 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed'

# output filepath prefix for pW_WC
o = ''
# o = 'LD_Fisher_vocab_in_Buckeye_contexts/LTR_Buckeye_aligned_CM_filtered_LM_filtered_in_buckeye_contexts.pW_WC'
# o = 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_WC'
o = 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts'

# for sanity checking...

# the sorted prefixes of W here correspond to the W dimension of file 'w'
p = ''
# p = 'LTR_Buckeye_aligned_w_GD_AmE_destressed/LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V.json'
p = 'LTR_newdic_destressed_aligned_w_GD_AmE_destressed/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered.pW_V.json'

# the sorted conditioning triphones and outcome response uniphones correspond to
# the dimensions of file 'c'
t = ''
# t = 'CM_AmE_destressed_aligned_w_LTR_Buckeye_pseudocount0.01/LTR_Buckeye_aligned_CM_filtered_LM_filtered_pY1X0X1X2.json'
t = 'CM_AmE_destressed_aligned_w_LTR_newdic_destressed_pseudocount0.01/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_p3Y1X012.json'

# the sorted projected contexts here correspond to the C dimension of file 'w'
s = ''
# s = 'LD_Fisher_vocab_in_Buckeye_contexts/LM_filtered_buckeye_contexts.txt'
s = 'LD_Fisher_vocab_in_swbd2003_contexts/LM_filtered_swbd2003_contexts.txt'

# calculation parameters
n = ''
n = '50'

k = ''
k = '2'

r = ''
r = 'False' #if 'False', only calculate p(\hat{W}|W = w, c), i.e. don't calculate p(\hat{W}|P = p, c)

e = ''
e = 'True' #if r='False' and e='True', only calculate p(\hat{W} = w*| W = w*, c) ∀w ∈ W


#the (optional) beginning and end of the block of contexts this notebook should calculate
cStart = ''
cEnd = ''

In [6]:
output_dir = path.dirname(o)
if not path.exists(output_dir):
    print('Making output path {0}'.format(output_dir))
    makedirs(output_dir)

In [7]:
if n == '':
    n = 50
else:
    n = int(n)

In [8]:
if k == '':
    k = 2
else:
    k = int(k)

In [9]:
if r == '' or r == 'False':
    r = False
#     raise Exception('Assuming only full wordforms have been produced is not supported currently.')
elif r == 'True':
    r = True
else:
    raise Exception(f"r must be one of {'','True','False'}, got '{r}' instead")

In [10]:
if not r and (e == '' or e == 'True'):
    e = True
#     raise Exception('Only calculating the posterior probability of the actual (correct) full intended wordform is not supported currently.')
elif r and (e == '' or e == 'True'):
    raise Exception("e can only be True if r is False")
elif e == 'False':
    e = False
else:
    raise Exception(f"e must be one of {'','True','False'}, got '{e}'")

In [11]:
# postview_fp = path.join(path.dirname(t), "p6Y0X01.json"); postview_fp

# preview_fp = path.join(path.dirname(t), "p3Y1X01.json"); preview_fp

In [ ]:
cStart = int(cStart)
cEnd = int(cEnd)

assert cStart <= cEnd, f"First context id of block to calculate be <= the context id of the end of the block, got {cStart} and {cEnd} instead."

# Imports / load data

In [12]:
import pickle

In [13]:
import numpy as np
import torch
import sparse

In [14]:
import tiledb

In [15]:
from probdist import *

In [16]:
from tqdm import tqdm, tqdm_gui, tqdm_notebook

In [17]:
from joblib import Parallel, delayed

J = -1
# J = 16
BACKEND = 'multiprocessing'
# BACKEND = 'loky'
V = 10
PREFER = 'processes'
# PREFER = 'threads'

def identity(x):
    return x

def par(gen_expr):
    return Parallel(n_jobs=J, backend=BACKEND, verbose=V, prefer=PREFER)(gen_expr)

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')
    if torch.cuda.device_count() > 1:
        print(torch.cuda.get_device_name(1))
        print('Memory Usage:')
        print('Allocated:', round(torch.cuda.memory_allocated(1)/1024**3,1), 'GB')
        print('Cached:   ', round(torch.cuda.memory_cached(1)/1024**3,1), 'GB')

Using device: cpu



In [19]:
gpu = torch.device('cuda')
cpu = torch.device('cpu')

my_device = cpu

In [20]:
cuda_ft = torch.cuda.FloatTensor
cuda_dt = torch.cuda.DoubleTensor

ft = torch.FloatTensor
dt = torch.DoubleTensor

my_ft = ft
my_dt = dt

# torch.set_default_tensor_type(my_ft)
torch.set_default_tensor_type(my_dt)

## Load/extract sanity-checking data

We want to be able to make queries, spot checks, and sanity checks. That means we want to be able to reference
 1. the set of strings constituting segmental wordforms and prefixes
 2. the source and channel alphabets
 3. the channel distribution's conditioning triphones ∩ the triphones in the lexicon = the triphones in the lexicon
 4.  

Segmental wordforms were necessary for 
 - the lexicon metadata calculation (step 4b)
 - the contextual distribution on segmental wordforms (step 4c)
 - the definition of the segmental sequence channel matrices (step 4d)
 
What does each use as input? (Pass that to this notebook.)

Each notebook uses ...pW_V.json (or something slightly downstream of that)

### Segmental lexicon, prefixes, inventory, and triphones

In [21]:
pW_V = condDistsAsProbDists(importProbDist(p))

Extract 
 - `orthographic vocabulary`
 - `segmental vocabulary`

In [22]:
Vs = set(pW_V.keys())
Ws = union(mapValues(lambda dist: set(conditions(dist)), 
                     pW_V).values())
len(Vs)
len(Ws)

9411

9172

In [23]:
wordlengthsInclEdges = set(len(ds2t(w)) for w in Ws)
wordlengthsInclEdges
numWordsOfExactlyLength = {l:len(wordformsOfLength(l, Ws, True)) for l in wordlengthsInclEdges}
numWordsOfExactlyLength

{4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19}

{4: 5,
 5: 173,
 6: 1266,
 7: 1790,
 8: 1646,
 9: 1328,
 10: 1052,
 11: 828,
 12: 508,
 13: 329,
 14: 150,
 15: 66,
 16: 24,
 17: 6,
 19: 1}

In [24]:
wordLengthRange = sorted(list(range(min(wordlengthsInclEdges),
                                    max(wordlengthsInclEdges)+1)))
wordLengthRange

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [25]:
wordlengthsNotIncludingEdges = {each-2 for each in wordlengthsInclEdges}
wordlengthsNotIncludingEdges

{2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17}

In [26]:
lengthFreqs = {l:len(wordformsAtLeastLlong(l, Ws, True)) for l in wordlengthsInclEdges}
lengthFreqs

{4: 9172,
 5: 9167,
 6: 8994,
 7: 7728,
 8: 5938,
 9: 4292,
 10: 2964,
 11: 1912,
 12: 1084,
 13: 576,
 14: 247,
 15: 97,
 16: 31,
 17: 7,
 19: 1}

Create mappings between `segmental wordforms` and `segmental prefixes`.

In [27]:
prefix_relation = set(union({(w,p) for p in getPrefixes(w)} for w in tqdm(Ws)))
len(prefix_relation)

100%|██████████| 9172/9172 [00:12<00:00, 764.09it/s] 


79727

Extract 
 - `segmental prefixes`

In [28]:
Ps = set(map(lambda pair: pair[1],
             prefix_relation))

Create 
 - `sorted` versions of the `segmental vocabulary` and `segmental prefixes`

In [29]:
Ws_t = tuple(sorted(list(Ws)))
Ps_t = tuple(sorted(list(Ps)))
num_wordforms = len(Ws_t)
num_prefixes = len(Ps_t)

num_wordforms
num_prefixes

9172

42231

Create mappings between `orthographic wordforms` and `segmental wordforms`.

In [30]:
v_to_Ws = mapValues(lambda dist: set(conditions(dist)),
                    pW_V)
V_W_relation = {(v,w) 
                for v in v_to_Ws 
                for w in v_to_Ws[v]}
w_to_Vs = {w:{v for v in Vs if (v,w) in V_W_relation}
           for w in Ws}

Extract
 - `segmental inventory`
 - `triphones` in the `segmental lexicon`

In [31]:
source_alphabet = lexiconToInventory(Ws)
Xs = source_alphabet
len(Xs)
Xs

41

{'aɪ',
 'aʊ',
 'b',
 'd',
 'dʒ',
 'eɪ',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'oʊ',
 'p',
 's',
 't',
 'tʃ',
 'u',
 'v',
 'w',
 'z',
 'æ',
 'ð',
 'ŋ',
 'ɑ',
 'ɔɪ',
 'ə',
 'ɚ',
 'ɛ',
 'ɪ',
 'ɹ',
 'ʃ',
 'ʊ',
 'ʌ',
 'ʒ',
 'θ',
 '⋉',
 '⋊'}

In [32]:
Xs_t = tuple(sorted(Xs))

In [33]:
lexicon_triphones = lexiconTo3factors(Ws)
len(lexicon_triphones)
list(lexicon_triphones)[:10]

7412

['aɪ.k.l',
 'b.m.ɪ',
 'u.t.eɪ',
 'w.eɪ.⋉',
 'ɪ.d.aɪ',
 'ʃ.æ.m',
 'ɪ.z.l',
 'b.u.n',
 'l.u.s',
 'ʊ.v.z']

In [34]:
X012s = lexicon_triphones
X012s_t = tuple(sorted(list(X012s)))

In [35]:
triphs_with_LE = {triph for triph in lexicon_triphones if leftEdge in triph}
len(triphs_with_LE)

489

In [36]:
triphs_with_RE = {triph for triph in lexicon_triphones if rightEdge in triph}
len(triphs_with_RE)

541

In [37]:
lexicon_diphones = lexiconTo2factors(Ws)
len(lexicon_diphones)
list(lexicon_diphones)[:10]

1003

['g.ʌ', 'ɑ.ŋ', 'ɪ.w', 'ɚ.ɚ', 'k.t', 'u.h', 'aɪ.⋉', 'b.w', 'aɪ.eɪ', '⋊.v']

In [38]:
X01s = lexicon_diphones
X01s_t = tuple(sorted(list(X01s)))

In [39]:
X01s_with_LE = {diph for diph in X01s if leftEdge in diph}
len(X01s_with_LE)

36

In [40]:
X01s_with_RE = {diph for diph in X01s if rightEdge in diph}
len(X01s_with_RE)

32

### Triphone channel distribution and channel alphabet

In [41]:
p3Y1X012 = condDistsAsProbDists(importProbDist(t))

assert uniformOutcomes(p3Y1X012)

In [42]:
channel_triphones = conditions(p3Y1X012)
len(channel_triphones)

assert all(triph in channel_triphones for triph in lexicon_triphones)

7412

In [43]:
{triph for triph in lexicon_triphones if triph not in channel_triphones}

set()

In [44]:
channel_alphabet = outcomes(p3Y1X012)
len(channel_alphabet)
channel_alphabet

39

{'aɪ',
 'aʊ',
 'b',
 'd',
 'dʒ',
 'eɪ',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'oʊ',
 'p',
 's',
 't',
 'tʃ',
 'u',
 'v',
 'w',
 'z',
 'æ',
 'ð',
 'ŋ',
 'ɑ',
 'ɔɪ',
 'ɚ',
 'ɛ',
 'ɪ',
 'ɹ',
 'ʃ',
 'ʊ',
 'ʌ',
 'ʒ',
 'θ',
 '⋉'}

In [45]:
Y1s = channel_alphabet
Y1s_t = tuple(sorted(Y1s))

In [46]:
in_lexicon_inventory_but_not_in_channel_inventory = source_alphabet - channel_alphabet
in_lexicon_inventory_but_not_in_channel_inventory

assert in_lexicon_inventory_but_not_in_channel_inventory == {leftEdge, rightEdge} or in_lexicon_inventory_but_not_in_channel_inventory == {'ə', leftEdge, rightEdge} or in_lexicon_inventory_but_not_in_channel_inventory == {'ə', leftEdge}

{'ə', '⋊'}

### Preview and postview distributions and alphabets

In [47]:
# preview_fp
# postview_fp

In [48]:
# p3Y1X01 = condDistsAsProbDists(importProbDist(preview_fp))

# p6Y0X01 = condDistsAsProbDists(importProbDist(postview_fp))

In [49]:
# assert uniformOutcomes(p3Y1X01)

# assert uniformOutcomes(p6Y0X01)

In [50]:
# preview_outcomes = outcomes(p3Y1X01)

# postview_outcomes = outcomes(p6Y0X01)

# assert preview_outcomes == Y1s

# assert postview_outcomes == Y1s

In [51]:
# preview_channel_diphones = conditions(p3Y1X01)
# len(preview_channel_diphones)

# # missing_preview_conditions = {diph for diph in lexicon_diphones if (diph not in preview_channel_diphones) and (leftEdge not in diph)}
# # missing_preview_conditions

# # p3Y1X01 = condProbDistAsDicts(p3Y1X01)

# # #add 𝚺⋉ to preview conditions
# # # for each 𝛔⋉ ∈ 𝚺⋉, ensure p(Y_1 = ⋉|𝛔⋉) = 1.0
# # for c in missing_preview_conditions:
# #     p3Y1X01[c] = {o:1.0 if ds2t(o)[1] == rightEdge else 0.0 for o in Y1s}

# # # add ⋉ to preview outcomes
# # for c in preview_channel_diphones
    
# # p3Y1X01 = condDistsAsProbDists(p3Y1X01)

# assert all(diph in preview_channel_diphones for diph in lexicon_diphones if not (leftEdge in diph or rightEdge in diph))

In [52]:
# postview_channel_diphones = conditions(p6Y0X01)
# len(postview_channel_diphones)

# assert all(diph in postview_channel_diphones for diph in lexicon_diphones if not (leftEdge in diph or rightEdge in diph))

In [53]:
# p3Y1X01 = condProbDistAsDicts(p3Y1X01)
# p6Y0X01 = condProbDistAsDicts(p6Y0X01)

# p3Y1X01 = {c:p3Y1X01[c] for c in lexicon_diphones if not (leftEdge in c or rightEdge in c)}
# p6Y0X01 = {c:p6Y0X01[c] for c in lexicon_diphones if not (leftEdge in c or rightEdge in c)}

# assert areNormalized(p3Y1X01)
# assert areNormalized(p6Y0X01)
# assert uniformOutcomes(p3Y1X01)
# assert uniformOutcomes(p6Y0X01)

# p3Y1X01 = condDistsAsProbDists(p3Y1X01)
# p6Y0X01 = condDistsAsProbDists(p6Y0X01)

In [54]:
# len(conditions(p3Y1X01))
# len(conditions(p6Y0X01))

In [55]:
# Y012s = set(map(t2ds, sigmaK(set.union(Y1s, edgeSymbols), 3)))
# len(set.union(Y1s, edgeSymbols))
# len(Y012s)
# Y012s_t = tuple(sorted(list(Y012s)))

### Corpus contexts

In [56]:
Cs = importSeqs(s, list)
len(Cs)

Cs_t = tuple(sorted(Cs))
Cs_t[123:129]

assert tuple(Cs) == Cs_t

106295

('<rem> and doctor',
 '<rem> and electric',
 '<rem> and especially',
 '<rem> and even',
 '<rem> and for',
 '<rem> and he')

### Conversion to one-hot vectors / sequences thereof

In [57]:
# construct what you need to convert to/from one-hot representations

# look at segment sequence channel matrix notebook

In [58]:
Xmap = seqsToIndexMap(Xs)
XOHmap = seqsToOneHotMap(Xs)

In [59]:
X012map = seqsToIndexMap(X012s)
# X012OHs = seqMapToOneHots(X012map)
X012OHmap = seqsToOneHotMap(X012s)

In [60]:
Y1map = seqsToIndexMap(Y1s)

In [61]:
def dsToUniphoneIndices(ds, uniphoneToIndexMap):
    uniphoneSeq = ds2t(ds)
    return np.array([uniphoneToIndexMap[uniphone] for uniphone in uniphoneSeq])

def dsToUniphoneOHs(ds, uniphoneToOHmap):
    uniphoneSeq = ds2t(ds)
    return np.array([uniphoneToOHmap[uniphone] for uniphone in uniphoneSeq])

def dsToTriphoneSeq(ds):
    return dsToKfactorSequence(3, ds)

def dsToTriphoneIndices(ds, triphoneToIndexMap):
    triphoneSeq = dsToTriphoneSeq(ds)
    return np.array([triphoneToIndexMap[triphone] for triphone in triphoneSeq])

def dsToTriphoneOHs(ds, triphoneToOHmap):
    triphoneSeq = dsToTriphoneSeq(ds)
    return np.array([triphoneToOHmap[triphone] for triphone in triphoneSeq])

dsToUniphoneIndices('t.i.f.l', Xmap)
dsToUniphoneOHs('t.i.f.l', XOHmap)
dsToTriphoneSeq('t.i.f.l')
dsToTriphoneIndices('t.i.f.l', X012map)
dsToTriphoneOHs('t.i.f.l', X012OHmap)
dsToTriphoneOHs('t.i.f.l', X012OHmap).shape
dsToTriphoneOHs('t.i.f.l', X012OHmap)[0].shape
dsToTriphoneOHs('t.i.f.l', X012OHmap)[0][5528]
dsToTriphoneOHs('t.i.f.l', X012OHmap)[1][5352]

array([18,  9,  6, 12])

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.]])

('t.i.f', 'i.f.l')

array([3756, 1449])

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

(2, 7412)

(7412,)

0.0

0.0

In [62]:
# Y1s_RE = outcomes(p3Y1X01)
# len(Y1s_RE)
# Y1s_RE_list = sorted(list(Y1s_RE))

# print(Y1s_RE - Y1s)

# Y1REmap = seqsToIndexMap(Y1s_RE)

# Y1REOHs = seqMapToOneHots(Y1REmap)
# Y1REOHmap = seqsToOneHotMap(Y1s_RE)
Y1OHmap = seqsToOneHotMap(Y1s)
# OHY1REmap = oneHotToSeqMap(Y1s_RE)
OHY1map = oneHotToSeqMap(Y1s)

def ymap(y):
#     return Y1REmap[y]
    return Y1map[y]

def y0kMap(y0k):
    return np.array(list(map(ymap, ds2t(y0k)[1:])))

def channelSeqOHs2ds(y0k_OHs, addLeftEdge = False):
    if not addLeftEdge:
#         return t2ds(tuple( map(OHY1REmap, tuple(y0k_OHs) ) ))
        return t2ds(tuple( map(OHY1map, tuple(y0k_OHs) ) ))
#     return leftEdge + '.' + t2ds(tuple( map(OHY1REmap, tuple(y0k_OHs) ) ))
    return leftEdge + '.' + t2ds(tuple( map(OHY1map, tuple(y0k_OHs) ) ))

def channelSeqds2OHs(y0k):
    y0k_t = ds2t(y0k)
    if leftEdge == y0k_t[0]:
        y1k_t = y0k_t[1:]
    y1k_t = y0k_t[1:]
#     return np.array([Y1REOHmap[ yj ] for yj in y1k_t]) #shape should be (_, 39)
    return np.array([Y1OHmap[ yj ] for yj in y1k_t]) #shape should be (_, 38)

def y0kOHmap(y0k):
##     y0k_t = ds2t(y0k)
## #     y0k_indices = y0kMap(y0k) #np.array(list(map(lambda y1: Y1map[y1], y0k_t[1:])))
##     y1k = t2ds(y0k_t[1:])
##     y0k_OHs = dsToUniphoneOHs(y1k, Y1REOHmap)
#     return dsToUniphoneOHs(t2ds(ds2t(y0k)[1:]), Y1REOHmap)
    y0k_t = ds2t(y0k) #let l = len(y0k_t)
    if y0k_t[0] == leftEdge:
        y1k_t = y0k_t[1:]
#         return np.array([Y1REOHmap[ yj ] for yj in y1k_t]) #shape should be (l-1, 39)
        return np.array([Y1OHmap[ yj ] for yj in y1k_t]) #shape should be (l-1, 38)
#     return np.array([Y1REOHmap[ yj ] for yj in y0k_t]) #shape should be (l, 39)
    return np.array([Y1OHmap[ yj ] for yj in y0k_t]) #shape should be (l, 38)


# list(Y1s_RE)[0]
list(Y1s)[0]
# OHY1REmap( Y1REOHmap[ list(Y1s_RE)[0] ] )
OHY1map( Y1OHmap[ list(Y1s)[0] ] )

channelSeqds2OHs(leftEdge + '.'+ 't.u.p')

channelSeqOHs2ds( channelSeqds2OHs(leftEdge + '.'+ 't.u.p') , True)

channelSeqOHs2ds(y0kOHmap(leftEdge + '.' + 't.u.p'), True)
channelSeqOHs2ds(y0kOHmap('t.u.p'))
len(ds2t(leftEdge + '.' + 't.u.p'))
len(ds2t('t.u.p'))
y0kOHmap('t.u.p').shape
y0kOHmap(leftEdge + '.' + 't.u.p').shape

'ɑ'

'ɑ'

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

'⋊.t.u.p'

'⋊.t.u.p'

't.u.p'

4

3

(3, 39)

(3, 39)

In [63]:
p3Y1X012_np = condDistFamilyToNP(p3Y1X012)
p3Y1X012_np.shape

(39, 7412)

In [64]:
from random import choice

In [65]:
random_source_wordform = choice(list(Ws))
random_source_wordform

random_source_prefix = choice(list(Ps))
random_source_prefix

'⋊.tʃ.ɛ.k.ɚ.⋉.⋉'

'⋊.b.aʊ.t.⋉.⋉'

In [66]:
def randomPrefix(l, alphabet=Xs):
    return randomString(alphabet, l, hasLeftEdge=True)

In [67]:
random_channel_prefix2 = randomPrefix(len(ds2t(random_source_wordform))-1, alphabet=Y1s)
random_channel_prefix2

'⋊.θ.θ.t.ɑ.n.oʊ'

In [68]:
# random_source_prefix = getRandomKey(pX0i)
random_source_prefix = choice(list(Ps))
while ds2t(random_source_prefix)[-1] == rightEdge:
#     random_source_prefix = getRandomKey(pX0i)
    random_source_prefix = choice(list(Ps))
while len(ds2t(random_source_prefix)) > len(ds2t(random_source_wordform)):
#     random_source_prefix = getRandomKey(pX0i)
    random_source_prefix = choice(list(Ps))
random_source_prefix
random_channel_prefix = randomPrefix(len(ds2t(random_source_prefix))-1, alphabet=Y1s)
random_channel_prefix

'⋊.ɪ.d.ɪ.ʃ.ɪ'

'⋊.g.æ.l.b.n'

In [69]:
def sourcePrefixToTriphones(x0k):
    xp_t = ds2t(x0k) #"x prefix"
    
#     xi = xp_t[-2] #just-completed segment
#     xk = xp_t[-1] #upcoming segment that we only get coarticulatory information about
    
#     xik_ds = t2ds((xi, xk))
#     preview_dist = p3Y1X01[xik_ds]
    
    x012s = dsToKfactorSequence(3, t2ds(xp_t))
    return x012s

random_triphoneSeq = sourcePrefixToTriphones(random_source_prefix)
random_triphoneSeq
threeFactorSequenceToDS(random_triphoneSeq)

('⋊.ɪ.d', 'ɪ.d.ɪ', 'd.ɪ.ʃ', 'ɪ.ʃ.ɪ')

'⋊.ɪ.d.ɪ.ʃ.ɪ'

In [70]:
def sourcePrefixToTriphoneIndices(x0k):
    triphoneSequence = sourcePrefixToTriphones(x0k)
    return tuple(map(lambda x012: X012map[x012], triphoneSequence))

sourcePrefixToTriphoneIndices(random_source_prefix)

(7349, 5826, 672, 6124)

## Load segmental sequence channel matrices $p(Y_0^k|X_0^k)$ and $p(Y_0^f|X_0^f)$

Index $i$ of `pY0k_X0ks` is a tensor of shape $|W| \times |Y_1| \times (i-2)$: for each wordform $w$ in $W$, there is a length $i-2$ stack of vectors giving the channel distribution for the length $i$ prefix (including the left word edge symbol and the upcoming segment) of $w$. (The stack is of length $i-2$ because there are $i-2$ triphones associated with a length $i$ source prefix.) Where necessary and appropriate (viz. when the wordform's length is incongruent w.r.t. the channel string), a wordform's channel matrix has been padded with columns of 0s to reflect the fact that the relevant source prefix could never have given rise to a channel string of that length.

`pY0f_X0fs` is an analogous tensor with the same shape, but intended for use in contexts where only wordforms whose length exactly matches the length of a channel string is worth considering as a cause of the channel string. (Accordingly, it has more zero vectors.)

In [71]:
pY0k_X0ks = pickle.load(open(c, 'rb'))
len(pY0k_X0ks)

20

In [72]:
pY0f_X0fs = pickle.load(open(f, 'rb'))
len(pY0f_X0fs)

20

In [73]:
pY0k_X0ks[0].dtype
pY0f_X0fs[0].dtype

dtype('float64')

dtype('float64')

In [74]:
wordlengthsNotIncludingEdges
wordlengthsInclEdges
lengthFreqs

{2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17}

{4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19}

{4: 9172,
 5: 9167,
 6: 8994,
 7: 7728,
 8: 5938,
 9: 4292,
 10: 2964,
 11: 1912,
 12: 1084,
 13: 576,
 14: 247,
 15: 97,
 16: 31,
 17: 7,
 19: 1}

In [75]:
len(Ws_t)
len(Y1s_t)

9172

39

In [76]:
pY0k_X0ks[0].shape
pY0k_X0ks[1].shape
pY0k_X0ks[2].shape
pY0k_X0ks[3].shape

pY0f_X0fs[0].shape
pY0f_X0fs[1].shape
pY0f_X0fs[2].shape
pY0f_X0fs[3].shape
pY0f_X0fs[4].shape

(0, 0)

(0, 0)

(0, 0)

(0, 0)

(0, 0)

(0, 0)

(0, 0)

(0, 0)

(9172, 39, 2)

In [77]:
pY0k_X0ks[4][0]
pY0f_X0fs[4][0]

array([[ 0.65264711,  0.02141693],
       [ 0.00700978,  0.00855777],
       [ 0.02545656,  0.00855777],
       [ 0.02545656,  0.58122414],
       [ 0.00682531,  0.00821919],
       [ 0.06235012,  0.00855777],
       [ 0.00682531,  0.00833257],
       [ 0.00700978,  0.02552273],
       [ 0.00682531,  0.00844442],
       [ 0.00700978,  0.00855777],
       [ 0.0051651 ,  0.00734595],
       [ 0.00682531,  0.00844442],
       [ 0.00700978,  0.00855777],
       [ 0.00700978,  0.00844442],
       [ 0.00700978,  0.01630829],
       [ 0.00700978,  0.00855777],
       [ 0.00700978,  0.00844442],
       [ 0.00682531,  0.00844442],
       [ 0.00700978,  0.05787536],
       [ 0.00700978,  0.00855777],
       [ 0.00700978,  0.00855777],
       [ 0.00682531,  0.00844442],
       [ 0.00664084,  0.00832952],
       [ 0.00682531,  0.00833257],
       [ 0.00700978,  0.00855777],
       [ 0.00682531,  0.00821919],
       [ 0.00700978,  0.00734595],
       [ 0.00700978,  0.00855777],
       [ 0.00700978,

array([[ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.],
       [ 0.,  0.]])

In [78]:
p3Y1X012_np.shape

(39, 7412)

In [79]:
pY0k_X0ks[4].shape
pY0f_X0fs[4].shape

(9172, 39, 2)

(9172, 39, 2)

In [80]:
def retrieve_CM_for(x0f_WE, total_prefix_length, exact = False):
    '''
    total_prefix_length includes 
     - the "⋊" symbol,
     - the actually and completely produced prefix,
     - the next (i.e. upcoming) segment.
     
    Ex:
      w = '⋊.aɪ.d.i.l.aɪ.z.⋉'
      |'⋊.aɪ.d.i.l.aɪ.z.⋉'| = 8
      l = 5
    =>
      'total_prefix' = '⋊.aɪ.d.i.l'
      |'⋊.aɪ.d.i.l'| = 5
      The associated channel matrix is of shape (|Y1s|, l-2 = 5-2 = 3).
    '''
    if not exact:
        return pY0k_X0ks[total_prefix_length][Ws_t.index(x0f_WE)]
    else:
        return pY0f_X0fs[total_prefix_length][Ws_t.index(x0f_WE)]

In [81]:
w_0 = Ws_t[0]; w_0
w_0_produced_prefix_noLE = t2ds(ds2t(Ws_t[0])[1:3+1]); w_0_produced_prefix_noLE, len(ds2t(w_0_produced_prefix_noLE))
w_0_produced_prefix = t2ds(ds2t(Ws_t[0])[0:3+1]); w_0_produced_prefix, len(ds2t(w_0_produced_prefix))
w_0_produced_prefix_plus_next = t2ds(ds2t(Ws_t[0])[0:3+2]); w_0_produced_prefix_plus_next, len(ds2t(w_0_produced_prefix_plus_next))

pY0k_X0ks[5][0].shape
# pY0k_X0ks[3][0] #assume the produced prefix is of length i+1=3+1=4 (not including left word edge)
pY0k_X0ks[5][0] # assume the produced prefix is of length i-2=3 (not including left word edge)
                # assume the produced prefix is of length i-1=4 (including left word edge)
                # assume the produced prefix + upcoming segment is of length i=5 (including left word edge)

w_0_whole_prefix = t2ds(ds2t(Ws_t[0])[0:3+2]); w_0_whole_prefix
w_0_whole_prefix_3factors = sourcePrefixToTriphones(w_0_whole_prefix); w_0_whole_prefix_3factors
w_0_whole_prefix_3factor_indices = sourcePrefixToTriphoneIndices(w_0_whole_prefix); w_0_whole_prefix_3factor_indices
w_0_prefix_CM = np.vstack([p3Y1X012_np[:,source_triph_idx] for source_triph_idx in w_0_whole_prefix_3factor_indices]).T; w_0_prefix_CM
w_0_prefix_CM.shape

assert np.allclose( w_0_prefix_CM, pY0k_X0ks[5][0] )
assert np.allclose( pY0k_X0ks[5][0], retrieve_CM_for(Ws_t[0], 5))

'⋊.aɪ.d.i.l.aɪ.z.⋉.⋉'

('aɪ.d.i', 3)

('⋊.aɪ.d.i', 4)

('⋊.aɪ.d.i.l', 5)

(39, 3)

...

'⋊.aɪ.d.i.l'

('⋊.aɪ.d', 'aɪ.d.i', 'd.i.l')

(6923, 5, 498)

array([[ 0.65264711,  0.02141693,  0.00703146],
       [ 0.00700978,  0.00855777,  0.00703146],
       [ 0.02545656,  0.00855777,  0.00703146],
       [ 0.02545656,  0.58122414,  0.00693832],
       [ 0.00682531,  0.00821919,  0.00675326],
       [ 0.06235012,  0.00855777,  0.00703146],
       [ 0.00682531,  0.00833257,  0.00684642],
       [ 0.00700978,  0.02552273,  0.00703146],
       [ 0.00682531,  0.00844442,  0.00693832],
       [ 0.00700978,  0.00855777,  0.73787478],
       [ 0.0051651 ,  0.00734595,  0.00603577],
       [ 0.00682531,  0.00844442,  0.00693832],
       [ 0.00700978,  0.00855777,  0.00703146],
       [ 0.00700978,  0.00844442,  0.00693832],
       [ 0.00700978,  0.01630829,  0.00703146],
       [ 0.00700978,  0.00855777,  0.00703146],
       [ 0.00700978,  0.00844442,  0.00693832],
       [ 0.00682531,  0.00844442,  0.00693832],
       [ 0.00700978,  0.05787536,  0.00703146],
       [ 0.00700978,  0.00855777,  0.00703146],
       [ 0.00700978,  0.00855777,  0.007

(39, 3)

**Notes:**

 - `|pY0k_X0ks|` = $max_l = |w|. ∄w'. |w'| > |w|$ (around 16-18, usually) 
 - `pY0k_X0ks[i]` = the lexical channel matrices for 'total prefixes' of length `i` (i.e. incl. word edges and upcoming segment)
 - `pY0k_X0ks[i]` =  the lexical channel matrices for prefixes of length `i-1` (i.e. *not* incl. word edges, but incl. upcoming segment)
 - `|pY0k_X0ks[i]|` = `(|W|, |Y1|, i-2)` = (segmental wordforms, Y1s, number of triphones)
 
(The same statements hold wrt `pY0fX0f`.)

## Load contextual distribution on segmental wordforms $p(W|C)$

In [82]:
pW_C = np.load(w)
pW_C.shape
pW_C.dtype
pW_C.nbytes / 1e9

(9172, 106295)

dtype('float64')

7.79950192

In [83]:
len(Ws_t)
len(Cs_t)

9172

106295

In [84]:
Cs_t[Cs_t.index('a couple of')]
pW_C[:,Cs_t.index('a couple of')]
isNormalized_np(pW_C[:,Cs_t.index('a couple of')])

'a couple of'

array([  2.52036749e-10,   1.50634197e-08,   2.52036749e-10, ...,
         5.54693797e-10,   4.15967189e-08,   6.22123621e-07])

True

In [85]:
Cs_t.index('a couple of')

1753

In [86]:
def pW_C_lookup(w=None,c=None):
    if w is None and c is None:
        raise Exception('Must specify at least one of a context string or segmental wordform string')
    if w is None:
        my_pW_c = pW_C[:,Cs_t.index(c)]
        my_pW_c_as_dict = dict(zip(Ws_t, my_pW_c))
        assert isNormalized(my_pW_c_as_dict)
        return ProbDist(my_pW_c_as_dict)
    if w is not None and c is not None:
        my_pw_c = pW_C[Ws_t.index(w), Cs_t.index(c)]
        return my_pw_c
    if c is None:
        my_pw_C = pW_C[Ws_t.index(w), :]
        my_pw_C_as_dict = dict(zip(Cs_t, my_pw_C))
        return my_pw_C_as_dict

In [87]:
my_dist = pW_C_lookup(c='a couple of')

In [88]:
my_dist.most_common(10)

[('⋊.t.aɪ.m.z.⋉.⋉', 0.70170688240694257),
 ('⋊.ð.ɛ.m.⋉.⋉', 0.058586970250377776),
 ('⋊.ð.ə.⋉.⋉', 0.054670419056498991),
 ('⋊.ʌ.ð.ɚ.⋉.⋉', 0.036645416028036247),
 ('⋊.m.aɪ.⋉.⋉', 0.028159696907432537),
 ('⋊.ð.oʊ.z.⋉.⋉', 0.020634293629461325),
 ('⋊.h.ʌ.n.d.ɹ.ə.d.⋉.⋉', 0.017489202057292059),
 ('⋊.l.aɪ.k.⋉.⋉', 0.014006091693604115),
 ('⋊.g.ʊ.d.⋉.⋉', 0.005842215226429354),
 ('⋊.m.ɪ.l.i.ə.n.⋉.⋉', 0.0053049068571393984)]

In [89]:
random_context = choice(Cs_t); random_context
Cs_t.index(random_context)
pW_C_lookup(c=random_context).most_common(10)

'her long-term'

28435

[('⋊.æ.n.s.ɚ.⋉.⋉', 0.3077004766624486),
 ('⋊.p.ɚ.s.p.ɛ.k.t.ɪ.v.⋉.⋉', 0.30751373602660675),
 ('⋊.æ.n.d.⋉.⋉', 0.034778653934661244),
 ('⋊.j.u.⋉.⋉', 0.015790462178778317),
 ('⋊.aɪ.⋉.⋉', 0.014558894265377069),
 ('⋊.oʊ.ɹ.⋉.⋉', 0.013894000089549031),
 ('⋊.ɪ.n.⋉.⋉', 0.011615695427060676),
 ('⋊.t.u.⋉.⋉', 0.011601352878858362),
 ('⋊.ð.æ.t.⋉.⋉', 0.010613629182021873),
 ('⋊.b.ʌ.t.⋉.⋉', 0.010353079352900975)]

## Load lexicon metadata

In [90]:
cousin_fn_map = {i:'{0}cousinsOf.npz'.format(i) 
                 for i in range(5)}
sphere_fn_map = {i:'{0}spheresOf.npz'.format(i) 
                 for i in range(5)}

In [91]:
cousin_fn_map

{0: '0cousinsOf.npz',
 1: '1cousinsOf.npz',
 2: '2cousinsOf.npz',
 3: '3cousinsOf.npz',
 4: '4cousinsOf.npz'}

In [92]:
sphere_fn_map

{0: '0spheresOf.npz',
 1: '1spheresOf.npz',
 2: '2spheresOf.npz',
 3: '3spheresOf.npz',
 4: '4spheresOf.npz'}

In [93]:
assert all(fn in listdir(m) for fn in cousin_fn_map.values())
assert all(fn in listdir(m) for fn in sphere_fn_map.values())

In [94]:
chdir(m)

In [95]:
cousin_mats = mapValues(sparse.load_npz, cousin_fn_map)

In [96]:
sphere_mats = mapValues(sparse.load_npz, sphere_fn_map)

In [97]:
chdir(repo_dir)

The $k$th-cousin/sphere matrix $m$ is a matrix of shape $|P| \times |W|$, where $m_{i,j}$ = 1 iff prefix $p_i$ is in a $k$-cousin/sphere relation to wordform $w_j$.

In [98]:
len(Ps_t)
len(Ws_t)

42231

9172

In [99]:
cousin_mats
mapValues(lambda m: m.nbytes / 1e9,
          cousin_mats)

{0: <COO: shape=(42231, 9172), dtype=uint8, nnz=79727, fill_value=0>,
 1: <COO: shape=(42231, 9172), dtype=uint8, nnz=1000076, fill_value=0>,
 2: <COO: shape=(42231, 9172), dtype=uint8, nnz=9307972, fill_value=0>,
 3: <COO: shape=(42231, 9172), dtype=uint8, nnz=39821468, fill_value=0>,
 4: <COO: shape=(42231, 9172), dtype=uint8, nnz=71449640, fill_value=0>}

{0: 0.001355359, 1: 0.017001292, 2: 0.158235524, 3: 0.676964956, 4: 1.21464388}

In [100]:
sphere_mats
mapValues(lambda m: m.nbytes / 1e9,
          sphere_mats)

{0: <COO: shape=(42231, 9172), dtype=uint8, nnz=9172, fill_value=0>,
 1: <COO: shape=(42231, 9172), dtype=uint8, nnz=30652, fill_value=0>,
 2: <COO: shape=(42231, 9172), dtype=uint8, nnz=370148, fill_value=0>,
 3: <COO: shape=(42231, 9172), dtype=uint8, nnz=2062370, fill_value=0>,
 4: <COO: shape=(42231, 9172), dtype=uint8, nnz=3207990, fill_value=0>}

{0: 0.000155924, 1: 0.000521084, 2: 0.006292516, 3: 0.03506029, 4: 0.05453583}

In [101]:
# c1.coords

In [102]:
c1 = cousin_mats[1]
(c1.coords[0][-2], c1.coords[1][-2])
Ps_t[c1.coords[0][-2]]
Ws_t[c1.coords[1][-2]]

(42229, 6949)

'⋊.θ.ʌ.n.d.ɚ.⋉'

'⋊.w.ʌ.n.d.ɚ.⋉.⋉'

In [103]:
m

'LTR_newdic_destressed_aligned_w_GD_AmE_destressed'

In [104]:
# segmental_wordforms = importSeqs(path.join(m, 'LTR_Buckeye_aligned_CM_filtered_LM_filtered.pW_V_Transcriptions.txt'))
# len(segmental_wordforms)

In [105]:
# list(segmental_wordforms)[0]

In [106]:
# Ws_t = tuple(sorted(list(map(padInputSequenceWithBoundaries,
#                              segmental_wordforms))))
# len(Ws_t)

In [107]:
# Ps = union(map(lambda w: getPrefixes(padInputSequenceWithBoundaries(w)), segmental_wordforms))
# len(Ps)

In [108]:
# Ps_t = tuple(sorted(list(Ps)))
# len(Ps_t)

# Slow, but sanity-checking calculations

In [109]:
# FIXME update this cell once you add in cells for doing calculations the old-fashioned way 
# and verifying/demonstrating that the calculation cell works correctly

## $p(C_0^i | X_0^i; X_{i+1})$

$C_i = (Y_{i-1}^{x_i}, Y_i^{x_i}, Y_{i+1}^{x_i})$

$p(y_{i-1}, y_i; y_{i+1} | x_{i-1}^i; x_{i+1}) = p(y_{i-1} | x_{i-1}, x_i;) p(y_i | x_{i-1}^i ; x_{i+1}) p(y_{i+1} | x_i ; x_{i+1})$

$p(c_0^i | x_0^i; x_{i+1}) = \prod\limits_{j=0}^{j=i} p(c_i | x_{i-1}^i ; x_{i+1})$

In [110]:
# def pC1_X012(y012, x012):
#     x012_t = ds2t(x012)
#     y012_t = ds2t(y012)
    
#     x0, x1, x2 = x012_t[0], x012_t[1], x012_t[2]
#     x01 = (x0, x1)
#     x12 = (x1, x2)
    
#     y0, y1, y2 = y012_t[0], y012_t[1], y012_t[2]
    
#     if x0 == leftEdge:
#         if y0 == leftEdge:
#             left_term = 1.0
#         else:
#             left_term = 0.0
#     else:
#         if y0 == leftEdge:
#             left_term = 0.0
#         else:
#             left_term = p6Y0X01[x01][y0]
    
#     center_term = p3Y1X012[x012][y1]
    
#     if x1 == rightEdge:
#         if y1 == rightEdge:
#             right_term = 1.0
#         else:
#             right_term = 0.0
#     else:
#         if y1 == rightEdge:
#             right_term = 0.0
#         else:
#             right_term = p3Y1X01[x12][y2]
    
#     terms = (left_term, center_term, right_term)
    
#     result = prod(terms)
    
#     return result

# def pC0iX0k(c0i, x0k):
#     xp_t = ds2t(x0k)
#     three_factor_seq = dsToKfactorSequence(3, x0k)
    
#     inputs = zip(c0i, three_factor_seq)
    
#     terms = starmap(pC1_X012, inputs)
    
#     result = prod(terms)
    
#     return result

In [111]:
# s = leftEdge + '.i.p.i.' + rightEdge; s
# dsToKfactorSequence(3, s)
# threeFactorSequenceToDS(dsToKfactorSequence(3, s))

In [112]:
# len(Y012s)
# len(X012s)
# len(Y012s) * len(X012s)

In [113]:
# # takes 5.5m on sidious and 45GB
# square = set(product(Y012s, X012s))

In [114]:
# def pY012_X012_calc(y012, x012):
#     return (x012, pC1_X012(y012, x012))

# pY012_X012 = dict(par(delayed(pY012_X012_calc)(y012, x012) for y012, x012 in product(Y012s, X012s)))

# # def pY012_X012_calc(y012_x012_pair):
# #     y012, x012 = y012_x012_pair
# #     return (x012, pC1_X012(y012, x012))

# # pY012_X012 = dict(par(delayed(pY012_X012_calc)(y012_x012_pair) for y012_x012_pair in product(Y012s, X012s)))

## $p(Y_0^i|X_0^k)$

In [115]:
len(pY0k_X0ks)
sorted(wordlengthsInclEdges)
assert len(pY0k_X0ks)-1 == sorted(wordlengthsInclEdges)[-1]

20

[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19]

In [116]:
len(Ws)
len(Ps)

9172

42231

In [117]:
pY0k_X0ks[3].shape

(0, 0)

In [118]:
def pY0iX0k(y0i, x0k, debug = False):
    xp_t = ds2t(x0k) #"x prefix"
    yp_t = ds2t(y0i) #"y prefix"
#     assert xp_t[0] == leftEdge
#     assert yp_t[0] == leftEdge
    if len(xp_t) < 3:
        raise Exception('|x0k| must be ≥ 3.')
    if (len(yp_t)+1 != len(xp_t)) and not ((yp_t[-1] == rightEdge) and len(yp_t) == len(xp_t)):
        raise Exception('|y0i| must = |x0k|-1, or |y0k| == |x0k| and y_k == ⋉.')
#     if len(yp_t) != len(xp_t):
#         raise Exception('Lengths of y0i and x0k must match.')
    # if len(xp_t) == 1:
    #     if xp_t[0] != leftEdge:
    #         raise Exception('First symbol of x0k must be left word edge symbol.')    
    #     if yp_t[0] == leftEdge:
    #         return 1.0
    #     else:
    #         return 0.0
    if yp_t[-1] == rightEdge:
        #if y_{f+1} == ⋉, then 
        # the only strings x_0^f that could generate it
        # have exactly the same length and also end with ⋉
        if len(yp_t) != len(xp_t):
            return 0.0
        
        x_k = xp_t[-1]
        if x_k != rightEdge:
            return 0.0
        
        #if |x_0^f| == |y_0^f| ∧ (y_{f+1} == x_{f+1} == ⋉),
        # then remove y_{f+1} and proceed with the calculation
        # as normal...
        y_k = rightEdge
        yp_t = yp_t[:-1]
    elif xp_t[-1] == rightEdge:
        # if x_k = ⋉ and y_k != ⋉, then the probability of that channel string is 0.0...
        return 0.0
    
#     xi = xp_t[-2] #just-completed segment
#     xk = xp_t[-1] #upcoming segment that we only get coarticulatory information about
    # yk = yp_t[-1]
    # xik_ds = t2ds((xi, xk))
    # preview_dist = p3Y1X01[xik_ds]
#     if yk not in preview_dist:
#         print('y0i: {0}\nx0k: {1}\nxi: {2}\nxk: {3}\nyk: {4}\nxik: {5}'.format(y0i, x0k, xi, xk, yk, xik_ds))
#     preview_term = preview_dist[yk]
    
#     if debug:
#         print('xi = {0}\nxk = {1}\nyk = {2}'.format(xi, xk, yk))
#         print('preview term = {0}'.format(preview_term))
    
#     x0i_t = xp_t[:-1] #prefix of intended wordform that has actually been produced
    # y0i_t = yp_t[:-1]
#     x0i_ds = t2ds(x0i_t)
#     y0i_ds = t2ds(y0i_t)

#     if debug:
#         x0i_t = xp_t[:-1] #prefix of intended wordform that has actually been produced
#         x0i_ds = t2ds(x0i_t)
#         y0i_ds = t2ds(y0i_t)
#         print('produced prefix x0i = {0}'.format(x0i_ds))
#         print('perceived prefix y0i = {0}'.format(y0i_ds))
    
#     x012s = dsToKfactorSequence(3, x0i_ds)
#     x012s = dsToKfactorSequence(3, t2ds(xp_t))
    x012s = dsToKfactorSequence(3, x0k)
    # y1s = y0i_t[1:]
    y1s = yp_t[1:]
#     pairs = list(zip(x012s, y1s))
#     produced_terms = tuple([p3Y1X012[x012][y1] for x012, y1 in pairs])
    pairs = zip(x012s, y1s)
    produced_terms = (p3Y1X012[x012][y1] for x012, y1 in pairs)
    
#     result = preview_term * prod(produced_terms)
    result = prod(produced_terms)

#     if debug:
#         print('x012s: {0}'.format(x012s))
#         print('y1s: {0}'.format(y1s))
#         print('main terms: {0}'.format(produced_terms))
#         print('end result = {0}'.format(result))
    
    if debug:
#         return result, [p3Y1X012[x012] for x012 in x012s]
        return result, np.array([[p3Y1X012[x012][k] for k in sorted(p3Y1X012[x012].keys())] for x012 in x012s]).T


    return result

In [122]:
w_0
# w_0_prefix = 
w_0_3 = t2ds(ds2t(w_0)[:3+2]); w_0_3# + 1
w_0_4 = t2ds(ds2t(w_0)[:4+2]); w_0_4# + 1
pY0k_X0ks[5][0]

'⋊.aɪ.d.i.l.aɪ.z.⋉.⋉'

'⋊.aɪ.d.i.l'

'⋊.aɪ.d.i.l.aɪ'

array([[ 0.65264711,  0.02141693,  0.00703146],
       [ 0.00700978,  0.00855777,  0.00703146],
       [ 0.02545656,  0.00855777,  0.00703146],
       [ 0.02545656,  0.58122414,  0.00693832],
       [ 0.00682531,  0.00821919,  0.00675326],
       [ 0.06235012,  0.00855777,  0.00703146],
       [ 0.00682531,  0.00833257,  0.00684642],
       [ 0.00700978,  0.02552273,  0.00703146],
       [ 0.00682531,  0.00844442,  0.00693832],
       [ 0.00700978,  0.00855777,  0.73787478],
       [ 0.0051651 ,  0.00734595,  0.00603577],
       [ 0.00682531,  0.00844442,  0.00693832],
       [ 0.00700978,  0.00855777,  0.00703146],
       [ 0.00700978,  0.00844442,  0.00693832],
       [ 0.00700978,  0.01630829,  0.00703146],
       [ 0.00700978,  0.00855777,  0.00703146],
       [ 0.00700978,  0.00844442,  0.00693832],
       [ 0.00682531,  0.00844442,  0.00693832],
       [ 0.00700978,  0.05787536,  0.00703146],
       [ 0.00700978,  0.00855777,  0.00703146],
       [ 0.00700978,  0.00855777,  0.007

In [123]:
pY0iX0k(w_0_3, w_0_4)
pY0iX0k(w_0_3, w_0_4, True)[1]
assert np.allclose( pY0k_X0ks[6][0], pY0iX0k(w_0_3, w_0_4, True)[1] )

0.1983147788326225

array([[ 0.65264711,  0.02141693,  0.00703146,  0.007226  ],
       [ 0.00700978,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.02545656,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.02545656,  0.58122414,  0.00693832,  0.00713029],
       [ 0.00682531,  0.00821919,  0.00675326,  0.00694011],
       [ 0.06235012,  0.00855777,  0.00703146,  0.007226  ],
       [ 0.00682531,  0.00833257,  0.00684642,  0.00703584],
       [ 0.00700978,  0.02552273,  0.00703146,  0.007226  ],
       [ 0.00682531,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.00855777,  0.73787478,  0.007226  ],
       [ 0.0051651 ,  0.00734595,  0.00603577,  0.00620276],
       [ 0.00682531,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.00855777,  0.00703146,  0.70851712],
       [ 0.00700978,  0.00844442,  0.00693832,  0.00713029],
       [ 0.00700978,  0.01630829,  0.00703146,  0.007226  ],
       [ 0.00700978,  0.00855777,  0.00703146,  0.02155085],
       [ 0.00700978,  0.

In [124]:
pY0iX0k('⋊.k.ɑ.n.t', '⋊.k.ɑ.n.t.⋉')
pY0iX0k('⋊.k.ɑ.n.t.⋉', '⋊.k.ɑ.n.t.⋉')

0.0

0.14201210183923127

In [125]:
pY0iX0k('⋊.k.ɑ.n', '⋊.k.ɑ.n.t')
pY0iX0k('⋊.k.ɑ.m', '⋊.k.ɑ.n.t')

0.25068286913932425

0.0034938697256087312

In [126]:
def trimToMatch(y0i, x0f):
    x0f_t = ds2t(x0f)
    y0i_t = ds2t(y0i)
    l = len(y0i_t)
    if len(x0f_t) < l+1:
        raise Exception('|x0f| must ≥ |y0i| + 1.')
    x0k_t = x0f_t[:l+1]
    x0k = t2ds(x0k_t)
    return x0k

def trimToLength(l, x0f, pad=False, padSymbol='?'):
    x0f_t = ds2t(x0f)
#     y0i_t = ds2t(y0i)
#     l = len(y0i_t)
    if len(x0f_t) < l and not pad:
        raise Exception('x0f must be at least as long as l.')
    if len(x0f_t) < l and pad:
        x0k_t = x0f_t[:l]
        x0k_t_padded = tuple( list(x0k_t) + [padSymbol] * (l - len(x0k_t)) )
        return t2ds(x0k_t_padded)
    else:
        x0k_t = x0f_t[:l]
        x0k = t2ds(x0k_t)
        return x0k
    
# trimToMatch('t.u.p', 't.u.p')
trimToMatch('t.u.p', 't.u.p.l')
trimToLength(3, 't.u.p')
trimToLength(3, 't.u.p.l')
trimToLength(6, 't.u.p.l', True)

't.u.p.l'

't.u.p'

't.u.p'

't.u.p.l.?.?'

In [127]:
def pY0iX0f(y0i, x0f, debug = False):
    x0f_t = ds2t(x0f)
    y0i_t = ds2t(y0i)
    l = len(y0i_t)
    if len(x0f_t) < l+1 and not ((y0i_t[-1] == rightEdge) and len(y0i_t) == len(x0f_t)):
        raise Exception('|x0f| must ≥ |y0i| + 1 or |x0f| = |y0k| ∧ y_k = ⋉')
    x0k_t = x0f_t[:l+1]
    x0k = t2ds(x0k_t)
#     print(y0i)
#     print(x0k)
    return pY0iX0k(y0i, x0k, debug)

In [130]:
pY0iX0f('⋊.k.ɑ.n', '⋊.k.ɑ.n.t')
pY0iX0f('⋊.k.ɑ.n', '⋊.k.ɑ.n.t.⋉')
pY0iX0f('⋊.k.ɑ.n.t', '⋊.k.ɑ.n.t.⋉')
pY0iX0f('⋊.k.ɑ.n.t', '⋊.k.ɑ.n.t.⋉')
pY0iX0f('⋊.k.ɑ.n.t.⋉', '⋊.k.ɑ.n.t.⋉.⋉')

0.25068286913932425

0.25068286913932425

0.0

0.0

0.0

In [131]:
random_x0f = choice(Ws_t); random_x0f
lw = len(ds2t(random_x0f))
while lw-1 <= 2:
    random_x0f = getRandomKey(pW); random_x0f
    lw = len(ds2t(random_x0f))

random_y0i = randomPrefix(choice(range(2,lw-1)), alphabet=Y1s); random_y0i
random_x0f
random_y0i
pY0iX0f(random_y0i, random_x0f)

'⋊.l.u.n.ɪ.t.ɪ.k.⋉.⋉'

'⋊.ɪ.dʒ.ʌ.i.l'

'⋊.l.u.n.ɪ.t.ɪ.k.⋉.⋉'

'⋊.ɪ.dʒ.ʌ.i.l'

1.0255586086802151e-10

## $p(X_0^f|C)$

In [132]:
pW_C.shape
(len(Ws), len(Cs))
assert pW_C.shape[0] == len(Ws)
assert pW_C.shape[1] == len(Cs)

(9172, 106295)

(9172, 106295)

In [133]:
Cs_t[Cs_t.index('a couple of')]

pX0f = pW_C[:,Cs_t.index('a couple of')]

pW = pW_C_lookup(c = Cs_t[Cs_t.index('a couple of')])

'a couple of'

In [134]:
def pX0f_C(w, c):
    return pW_C[:, Cs_t.index(c)]

## $p(Y_0^i | c)$

In [135]:
def pXhat0fY0i_unnormalized(xhat0f, y0i, c):
    likelihood = pY0iX0f(y0i, xhat0f)
#     prior = pW[xhat0f]
    prior = pW_C_lookup(xhat0f, c)
    return likelihood * prior

def pY0i(y0i, c):
#     l = len(ds2t(y0i))
    y0i_t = ds2t(y0i)
    l = len(y0i_t)
    if y0i_t[-1] == rightEdge:
        possible_source_wordforms = {w for w in Ws if len(ds2t(w)) == l}
    else:
        possible_source_wordforms = {w for w in Ws if len(ds2t(w)) >= l+1}
#     x0ksWithLengthl = prefixesWithLength(l)
#     py0kx0ks = tuple(pXhat0fY0k_unnormalized(x0k, y0k) for x0k in x0ksWithLengthl)

#     sufficiently_long_words = {w for w in Ws if len(ds2t(w)) >= l+1}
#     print(len(sufficiently_long_words))
    #line below is slower by about a third (pypy/kotoba)
#     sufficiently_long_words = wordformsAtLeastLlong(l, True)

    py0ix0fs = (pXhat0fY0i_unnormalized(x0f, y0i, c) for x0f in possible_source_wordforms)
#     def foo(x0f):
#         return pXhat0fY0k_unnormalized(x0f, y0k)
    # there's no apparent benefit to using foo + generator
#     py0kx0fs = (foo(x0f) for x0f in sufficiently_long_words)
# Parallel(n_jobs=-1, verbose=50, batch_size=4, prefer="processes", backend="multiprocessing")
#     py0kx0fs = Parallel(n_jobs=8, batch_size=4, verbose=50, prefer="threads", backend="multiprocessing")(delayed(pXhat0fY0k_unnormalized)(x0f, y0k) for x0f in sufficiently_long_words)
#     py0kx0fs = Parallel(n_jobs=8, prefer="threads")(delayed(pXhat0fY0k_unnormalized)(x0f, y0k) for x0f in sufficiently_long_words)
    return sum(py0ix0fs)

In [136]:
random_context = choice(Cs_t); random_context
pY0i('⋊.k.ɑ.n.t', random_context)

'well why are'

2.4897613275031891e-09

In [137]:
{w for w in Ws if (w,'⋊.k.ɑ.n.t') in prefix_relation}

{'⋊.k.ɑ.n.t.n.u.ɪ.t.i.⋉.⋉',
 '⋊.k.ɑ.n.t.n.ɪ.n.t.⋉.⋉',
 '⋊.k.ɑ.n.t.æ.k.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ə.m.p.l.eɪ.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɛ.k.s.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɛ.s.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.k.t.ɚ.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.k.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.s.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.b.j.u.ʃ.ɪ.n.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.b.æ.n.d.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.d.ɪ.k.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.v.ɚ.s.i.⋉.⋉'}

In [143]:
random_context = choice(Cs_t); random_context
pXhat0fY0i_unnormalized('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context)

'got to the'

1.0254745911355696e-08

## $p(Y_0^i|x_0^k)$ sampler

In [144]:
def sample_pY0iX0k(x0k, debug = False):
    xp_t = ds2t(x0k) #"x prefix"
#     yp_t = ds2t(y0i) #"y prefix"
#     if len(yp_t) != len(xp_t):
#         raise Exception('Lengths of y0i and x0i must match.')
    if len(xp_t) == 1:
        if xp_t[0] != leftEdge:
            raise Exception('First symbol of x0k must be left word edge symbol.')
        else:
            return leftEdge
    if len(xp_t) < 3:
        raise Exception('|x0k| must be ≥ 3')
    
    # xi = xp_t[-2] #just-completed segment
    # xk = xp_t[-1] #upcoming segment that we only get coarticulatory information about
    # xik = t2ds((xi, xk))
    # yk = sampleFrom(p3Y1X01[xik])
#     yk = yp_t[-1]
#     preview_term = p3Y1X01[t2ds((xi, xk))][yk]
    
#     if debug:
#         print('xi = {0}\nxk = {1}\nyk = {2}'.format(xi, xk, yk))
#         print('preview term = {0}'.format(preview_term))
    
#     x0i_t = xp_t[:-1] #prefix of intended wordform that has actually been produced
#     y0i_t = yp_t[:-1]
#     x0i_ds = t2ds(x0i_t)
#     y0i_ds = t2ds(y0i_t)

#     if debug:
#         print('produced prefix x0i = {0}'.format(x0i_ds))
#         print('perceived prefix y0i = {0}'.format(y0i_ds))
    
    x012s = dsToKfactorSequence(3, t2ds(xp_t))
#     print('x012s: {0}'.format(x012s))
#     y1s = y0i_t[1:]
    y1s = [sampleFrom(p3Y1X012[x012]) for x012 in x012s]
#     print('y1s: {0}'.format(y1s))
#     pairs = list(zip(x012s, y1s))
#     produced_terms = tuple([p3Y1X012[x012][y1] for x012, y1 in pairs])
    y0i_t = tuple( [leftEdge] + y1s )
#     print('y0i_t = {0}'.format(y0i_t))
    y0i_ds = t2ds(y0i_t)
#     print('y0i_ds = {0}'.format(y0i_ds))
#     assert len(y0i_t) == len(x0i_t)
    # yp_t = tuple([leftEdge] + y1s + [yk])
    # y0i = t2ds(yp_t)
    
    if xp_t[-1] == rightEdge:
        y0i_ds = y0i_ds + '.' + rightEdge
    
    if debug:
#         sorted_outcomes = Y1s_t
        return y0i_ds, np.array([[p3Y1X012[x012][k] for k in Y1s_t] for x012 in x012s]).T
#         return y0i_ds, np.array([[p3Y1X012[x012][k] for k in sorted(p3Y1X012[x012].keys())] for x012 in x012s]).T
#         print('x012s: {0}'.format(x012s))
#         print('y1s: {0}'.format(tuple(y1s)))
#         print('main terms: {0}'.format(produced_terms))

    return y0i_ds
#     return preview_term * prod(produced_terms)

In [145]:
'⋊.æ.k.t.ɪ.v.ə.t.i.⋉.⋉'
[sample_pY0iX0k('⋊.æ.k.t.ɪ.v.ə.t.i.⋉.⋉') for each in range(20)]
[sample_pY0iX0k('⋊.æ.k.t.ɪ.v.ə.t.i.⋉') for each in range(20)]
[sample_pY0iX0k('⋊.æ.k.t.ɪ.v.ə.t.i') for each in range(5)]

'⋊.æ.k.t.ɪ.v.ə.t.i.⋉.⋉'

['⋊.ŋ.oʊ.t.ʊ.v.i.t.i.⋉.⋉',
 '⋊.ʊ.h.ŋ.ɛ.v.ɛ.d.i.⋉.⋉',
 '⋊.ʊ.k.t.ɪ.f.g.θ.p.⋉.⋉',
 '⋊.ɑ.k.t.eɪ.v.aɪ.aʊ.ɛ.⋉.⋉',
 '⋊.k.k.t.ɪ.m.ɔɪ.h.i.⋉.⋉',
 '⋊.aɪ.k.t.ɪ.aʊ.ʌ.t.i.⋉.⋉',
 '⋊.m.k.t.s.ɑ.ɔɪ.i.i.⋉.⋉',
 '⋊.æ.k.aʊ.ʌ.f.ʌ.t.i.⋉.⋉',
 '⋊.aɪ.k.t.b.ʒ.ʌ.t.i.⋉.⋉',
 '⋊.aɪ.ʌ.i.ʒ.v.ɛ.t.i.⋉.⋉',
 '⋊.æ.k.t.ɪ.v.ʊ.dʒ.i.⋉.⋉',
 '⋊.ɑ.ŋ.t.z.v.g.b.i.⋉.⋉',
 '⋊.ɑ.g.t.ɪ.v.ʌ.t.ɛ.⋉.⋉',
 '⋊.b.j.t.v.v.j.t.i.⋉.⋉',
 '⋊.aɪ.k.s.ɪ.v.ʊ.p.i.⋉.⋉',
 '⋊.æ.k.t.ɪ.v.ʌ.t.h.⋉.⋉',
 '⋊.ɑ.æ.t.ʌ.ʊ.ʌ.t.i.⋉.⋉',
 '⋊.aɪ.k.t.b.v.ʌ.t.aɪ.⋉.⋉',
 '⋊.æ.k.t.ɪ.v.ɪ.t.i.⋉.⋉',
 '⋊.aɪ.k.t.ʊ.v.aɪ.t.p.⋉.⋉']

['⋊.ɑ.k.t.ɪ.v.k.dʒ.ɑ.⋉',
 '⋊.eɪ.k.tʃ.ɪ.m.ʌ.t.i.⋉',
 '⋊.ɑ.k.t.ɪ.v.t.t.i.⋉',
 '⋊.p.k.tʃ.ɪ.v.ʌ.t.i.⋉',
 '⋊.ɑ.ɹ.t.ɪ.v.eɪ.t.i.⋉',
 '⋊.eɪ.g.t.ɪ.v.θ.t.i.⋉',
 '⋊.æ.k.w.ɪ.ɛ.t.t.i.⋉',
 '⋊.aɪ.k.t.ɪ.ɹ.ʌ.t.i.⋉',
 '⋊.æ.w.t.ɪ.v.eɪ.t.i.⋉',
 '⋊.aɪ.k.ʒ.ɪ.v.ɪ.t.i.⋉',
 '⋊.ɑ.k.t.ɪ.dʒ.ð.t.i.⋉',
 '⋊.ɑ.k.t.ɪ.v.ð.z.z.⋉',
 '⋊.ɑ.ɪ.t.ʌ.ʊ.ʌ.t.i.⋉',
 '⋊.l.ŋ.k.ɪ.v.ð.t.i.⋉',
 '⋊.æ.θ.t.oʊ.v.ð.ʒ.s.⋉',
 '⋊.b.ɔɪ.tʃ.v.v.ʌ.t.i.⋉',
 '⋊.b.k.t.ɪ.v.f.t.tʃ.⋉',
 '⋊.p.k.u.ʌ.oʊ.ʌ.t.i.⋉',
 '⋊.æ.oʊ.t.eɪ.v.eɪ.dʒ.i.⋉',
 '⋊.p.k.t.ɪ.v.eɪ.t.i.⋉']

['⋊.v.k.dʒ.ɪ.v.θ.t',
 '⋊.aɪ.k.j.ɪ.v.s.t',
 '⋊.æ.k.t.ɪ.æ.d.t',
 '⋊.ɑ.k.aʊ.ɔɪ.v.ʌ.ŋ',
 '⋊.ɑ.k.p.ɪ.v.ɛ.t']

In [146]:
random_source_prefix
random_channel_samples = [sample_pY0iX0k(random_source_prefix) for each in range(50)]; random_channel_samples

'⋊.ɪ.d.ɪ.ʃ.ɪ'

['⋊.ɪ.d.ɪ.ʃ',
 '⋊.ɪ.d.ɪ.ʃ',
 '⋊.ɪ.ð.θ.ʃ',
 '⋊.ɪ.d.ɪ.ʃ',
 '⋊.m.d.ɪ.aʊ',
 '⋊.ɪ.d.ɪ.ʃ',
 '⋊.ɪ.ɛ.ɪ.ʃ',
 '⋊.ʒ.h.ɪ.ʃ',
 '⋊.ɚ.t.t.ʃ',
 '⋊.tʃ.θ.ɪ.ʃ',
 '⋊.eɪ.d.ɪ.p',
 '⋊.ɪ.d.ɪ.ʃ',
 '⋊.eɪ.n.ɪ.θ',
 '⋊.j.b.z.ʃ',
 '⋊.ɪ.d.ɪ.ʃ',
 '⋊.ɪ.d.k.v',
 '⋊.ɪ.dʒ.ɪ.ʃ',
 '⋊.ɪ.p.ɪ.ʃ',
 '⋊.ʌ.d.ɛ.ʃ',
 '⋊.ɪ.d.ɪ.θ',
 '⋊.z.d.ɪ.ʃ',
 '⋊.eɪ.d.ɪ.ʃ',
 '⋊.ɪ.d.b.θ',
 '⋊.ɛ.t.b.ʃ',
 '⋊.ɪ.b.ɪ.v',
 '⋊.m.t.ɪ.ʃ',
 '⋊.ɪ.d.ɪ.ʃ',
 '⋊.ɪ.ʒ.w.ʌ',
 '⋊.ɹ.t.ɪ.ʃ',
 '⋊.ɪ.d.ɪ.ʃ',
 '⋊.ɪ.d.ɪ.ʃ',
 '⋊.ɪ.z.ʃ.b',
 '⋊.ɪ.d.s.ʃ',
 '⋊.ɛ.d.ɪ.f',
 '⋊.ɪ.ɔɪ.ɪ.ʃ',
 '⋊.ɪ.d.ɪ.ʃ',
 '⋊.ɪ.s.i.ʃ',
 '⋊.ɪ.d.tʃ.j',
 '⋊.ɪ.d.ɪ.ʃ',
 '⋊.ɪ.d.j.ʃ',
 '⋊.ɪ.eɪ.ɪ.p',
 '⋊.ɪ.d.ɪ.ʃ',
 '⋊.ɪ.ŋ.ɪ.ʃ',
 '⋊.ɪ.d.d.ʃ',
 '⋊.ɪ.dʒ.ɪ.i',
 '⋊.æ.d.ɪ.k',
 '⋊.ɪ.d.ɪ.f',
 '⋊.ŋ.d.ɪ.ʃ',
 '⋊.ɪ.d.ɪ.ʃ',
 '⋊.ɪ.ʊ.ɪ.t']

In [147]:
sample_pY0iX0k('⋊.k.ɑ.n.t')

'⋊.k.s.n'

In [148]:
[sample_pY0iX0k('⋊.k.ɑ.n.t') for each in range(50)]

['⋊.k.ɑ.n',
 '⋊.k.ɚ.n',
 '⋊.l.ɑ.ʌ',
 '⋊.k.ɑ.n',
 '⋊.k.ʌ.dʒ',
 '⋊.k.z.n',
 '⋊.d.ɑ.n',
 '⋊.k.ʌ.n',
 '⋊.k.l.n',
 '⋊.aɪ.g.n',
 '⋊.s.ɑ.n',
 '⋊.k.ɹ.p',
 '⋊.k.ɪ.n',
 '⋊.k.s.n',
 '⋊.k.ɑ.n',
 '⋊.aɪ.ɑ.n',
 '⋊.dʒ.ɑ.n',
 '⋊.k.s.aɪ',
 '⋊.k.aɪ.i',
 '⋊.k.ɑ.h',
 '⋊.k.ʌ.dʒ',
 '⋊.ʊ.ɑ.n',
 '⋊.k.ɑ.tʃ',
 '⋊.k.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.eɪ.oʊ.n',
 '⋊.f.ɑ.n',
 '⋊.k.ɑ.ɹ',
 '⋊.ɪ.t.n',
 '⋊.b.ɑ.h',
 '⋊.j.ʌ.n',
 '⋊.oʊ.eɪ.n',
 '⋊.aɪ.ɑ.u',
 '⋊.k.ɑ.n',
 '⋊.k.ɑ.n',
 '⋊.k.ʃ.n',
 '⋊.ɚ.ɑ.b',
 '⋊.k.ʌ.n',
 '⋊.k.m.n',
 '⋊.k.ɑ.oʊ',
 '⋊.h.s.l',
 '⋊.k.oʊ.u',
 '⋊.k.ɑ.v',
 '⋊.k.ɑ.n',
 '⋊.k.ʃ.n',
 '⋊.k.ɑ.n',
 '⋊.ɹ.v.n',
 '⋊.k.ɑ.ɛ',
 '⋊.aɪ.ɑ.ɹ',
 '⋊.n.ɑ.n']

In [149]:
sample_pY0iX0k('⋊.k.ɑ.n.t', debug=True)[1].shape
sample_pY0iX0k('⋊.k.ɑ.n.t', debug=True)[1]

(39, 3)

array([[ 0.04390334,  0.01805178,  0.00911065],
       [ 0.00700978,  0.01374583,  0.00911065],
       [ 0.02545656,  0.00721312,  0.00911065],
       [ 0.00700978,  0.00711758,  0.00898997],
       [ 0.00682531,  0.00692774,  0.01700733],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.00682531,  0.0070233 ,  0.0088709 ],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.00682531,  0.00711758,  0.00898997],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.0051651 ,  0.00619171,  0.00782054],
       [ 0.65246265,  0.00711758,  0.00898997],
       [ 0.02545656,  0.00721312,  0.00911065],
       [ 0.00700978,  0.00711758,  0.00898997],
       [ 0.00700978,  0.00721312,  0.6450247 ],
       [ 0.00700978,  0.01374583,  0.00911065],
       [ 0.00700978,  0.00711758,  0.00898997],
       [ 0.00682531,  0.00711758,  0.00898997],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.00700978,  0.00721312,  0.009

## $p(\widehat{X_0^f}|Y_0^{i}, C)$

$p(\widehat{x_0^f}|y_0^{i}, c) = \frac{p(y_0^{i} | \widehat{x_0^f})p(\widehat{x_0^f} | c)}{p(y_0^{i})}$

In [150]:
def pXhat0fY0i_old(xhat0f, y0i, c):
    numerator = pXhat0fY0i_unnormalized(xhat0f, y0i, c)
    denominator = pY0i(y0i, c)
    if denominator == 0.0:
#         if numerator == 0.0:
#             print('both numerator and denominator are 0')
#         else:
#             print('just the denominator is 0')
        return 0
    return numerator / denominator

random_context = choice(Cs_t); random_context
(random_source_wordform, random_channel_prefix, random_context)
pXhat0fY0i_old(random_source_wordform, random_channel_prefix, random_context)

'well maybe some'

('⋊.tʃ.ɛ.k.ɚ.⋉.⋉', '⋊.g.æ.l.b.n', 'well maybe some')

0.0

## $\widehat{p}(\widehat{X_0^f}|X_0^i;X_{i+1}, C)$

$p(\widehat{x_0^f} | x_0^i;x_{i+1}, c) = \sum\limits_{y_0^{i}} p(\widehat{x_0^f} | y_0^{i}, c)p(y_0^{i}|x_0^{i};x_{i+1}, c)$

$\widehat{p}(\widehat{x_0^f}|x_0^i;x_{i+1}, c) = \frac{1}{m}\sum\limits_{m \text{ sampled } y_0^{i}\text{s from } p(Y_0^{i} | x_0^{i};x_{i+1})} p(\widehat{x_0^f} | y_0^{i}, c)$

In [151]:
# BASELINE = OLD
def phatXhat0fX0k_baseline(xhat0f, x0k, c, m = 50, my_j=None):
#     y0ks = tuple(sample_pY0kX0k(x0k) for each in range(n))
#     terms = (pXhat0fY0k(xhat0f, y0k) for y0k in y0ks)

#     foo = partial(pXhat0fY0i, xhat0f)
#     def foo(y0i):
#         return pXhat0fY0i(xhat0f, y0i)
#     terms = (foo(y0i) for y0i in (sample_pY0iX0i(x0i) for each in range(n)))
#     terms = Parallel(n_jobs=8, prefer="threads")(delayed(foo)(y0i) for y0i in y0is)
#     terms = Parallel(n_jobs=8, prefer="threads")(delayed(foo)(y0i) for y0i in (sample_pY0iX0i(x0i) for each in range(n)))
#     terms = Parallel(n_jobs=5, verbose=50, batch_size=10, prefer="threads")(delayed(pXhat0fY0i)(xhat0f, y0i) for y0i in (sample_pY0iX0i(x0i) for each in range(n)))
#     terms = Parallel(n_jobs=-1, verbose=50, batch_size=round(n/12), prefer="processes", backend="multiprocessing")(delayed(pXhat0fY0i)(xhat0f, y0i) for y0i in (sample_pY0iX0i(x0i) for each in range(n)))

#     terms = Parallel(n_jobs=-1, batch_size=round(n/12), prefer="processes", backend="multiprocessing")(delayed(pXhat0fY0k_old)(xhat0f, y0k) for y0k in (sample_pY0kX0k(x0k) for each in range(n)))
#     terms = Parallel(n_jobs=-1, batch_size='auto', prefer="processes", backend="multiprocessing")(delayed(pXhat0fY0i_old)(xhat0f, y0i) for y0i in (sample_pY0iX0k(x0k) for each in range(m)))
    if my_j is None:
        my_j = -1
    terms = Parallel(n_jobs=my_j, batch_size='auto', prefer="processes", backend="multiprocessing")(delayed(pXhat0fY0i_old)(xhat0f, y0i, c) for y0i in (sample_pY0iX0k(x0k) for each in range(m)))
#     terms = (pXhat0fY0k_old(xhat0f, y0k) for y0k in (sample_pY0kX0k(x0k) for each in range(n)))
    s = sum(terms)
    sbar = (1.0 * s) / m
    return sbar

In [152]:
random_context = choice(Cs_t); random_context

'confirm'

In [153]:
phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 200)

0.03366498462529037

In [154]:
phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 50)

0.034020414971244195

In [155]:
{w for w in Ws if (w,'⋊.k.ɑ.n.t') in prefix_relation}
[f"p(W' = {w}) → {pW[w]}"
 for w in sorted(list({w for w in Ws if (w,'⋊.k.ɑ.n.t') in prefix_relation}),
                 key=lambda w:pW_C_lookup(w=w, c=random_context), reverse=True)]

[f"p(W' = {w}|r = '⋊.k.ɑ.n.t') → {phatXhat0fX0k_baseline(w, '⋊.k.ɑ.n.t', random_context, 50, -1)}"
 for w in sorted(list({w for w in Ws if (w,'⋊.k.ɑ.n.t') in prefix_relation}),
                 key=lambda w:phatXhat0fX0k_baseline(w, '⋊.k.ɑ.n.t', random_context, 50, -1), reverse=True)]

{'⋊.k.ɑ.n.t.n.u.ɪ.t.i.⋉.⋉',
 '⋊.k.ɑ.n.t.n.ɪ.n.t.⋉.⋉',
 '⋊.k.ɑ.n.t.æ.k.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ə.m.p.l.eɪ.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɛ.k.s.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɛ.s.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.k.t.ɚ.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.k.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.s.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.b.j.u.ʃ.ɪ.n.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.b.æ.n.d.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.d.ɪ.k.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.v.ɚ.s.i.⋉.⋉'}

["p(W' = ⋊.k.ɑ.n.t.æ.k.t.⋉.⋉) → 1.9927084903616688e-06",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.æ.k.t.⋉.⋉) → 1.1031811727794093e-07",
 "p(W' = ⋊.k.ɑ.n.t.ɛ.s.t.⋉.⋉) → 1.7458090784748148e-07",
 "p(W' = ⋊.k.ɑ.n.t.ɛ.k.s.t.⋉.⋉) → 3.301887226828777e-07",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.v.ɚ.s.i.⋉.⋉) → 9.314244854919203e-07",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.b.j.u.ʃ.ɪ.n.⋉.⋉) → 6.6987445094223545e-09",
 "p(W' = ⋊.k.ɑ.n.t.n.ɪ.n.t.⋉.⋉) → 4.2201127768971105e-09",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.æ.k.t.ɚ.⋉.⋉) → 3.6577460887984323e-09",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.æ.s.t.⋉.⋉) → 3.1576348202842305e-08",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.d.ɪ.k.t.⋉.⋉) → 2.946759813831917e-08",
 "p(W' = ⋊.k.ɑ.n.t.ə.m.p.l.eɪ.t.⋉.⋉) → 8.876113708312564e-10",
 "p(W' = ⋊.k.ɑ.n.t.n.u.ɪ.t.i.⋉.⋉) → 5.546937969092835e-10",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.b.æ.n.d.⋉.⋉) → 2.5203674880187e-10"]

["p(W' = ⋊.k.ɑ.n.t.æ.k.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.0267914660520877",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.æ.k.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.012883211506386383",
 "p(W' = ⋊.k.ɑ.n.t.ɛ.s.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.007172572131121353",
 "p(W' = ⋊.k.ɑ.n.t.ɛ.k.s.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.006263952455516425",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.v.ɚ.s.i.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.004678263538411015",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.b.j.u.ʃ.ɪ.n.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.001960905846523982",
 "p(W' = ⋊.k.ɑ.n.t.n.ɪ.n.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.0016844925800000201",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.æ.s.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.0007503120357578348",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.æ.k.t.ɚ.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.0009630028002493904",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.d.ɪ.k.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.0004576751860343127",
 "p(W' = ⋊.k.ɑ.n.t.ə.m.p.l.eɪ.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.0002501354301014082",
 "p(W' = ⋊.k.ɑ.n.t.n.u.ɪ.t.i.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.00017215768795695785",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.b.æ.n.d.⋉.⋉|r = '⋊.k.ɑ.n.t') → 5.377792943151977e-05"]

In [156]:
# {w for w in Ws if (w,'⋊.k.ɑ.n') in prefix_relation}

[f"p(W' = {w}) → {pW_C_lookup(w=w, c=random_context)}"
 for w in sorted(list({w for w in Ws if (w,'⋊.k.ɑ.n') in prefix_relation}),
                 key=lambda w:pW_C_lookup(w=w, c=random_context), reverse=True)]

[f"p(W' = {w}|r = '⋊.k.ɑ.n.t') → {phatXhat0fX0k_baseline(w, '⋊.k.ɑ.n.t', random_context, 50, -1)}"
 for w in sorted(list({w for w in Ws if (w,'⋊.k.ɑ.n') in prefix_relation}),
                 key=lambda w:pW_C_lookup(w=w, c=random_context), reverse=True)]

["p(W' = ⋊.k.ɑ.n.v.ɚ.s.eɪ.ʃ.ɪ.n.⋉.⋉) → 2.0399590263267035e-05",
 "p(W' = ⋊.k.ɑ.n.t.æ.k.t.⋉.⋉) → 1.3450859684694053e-05",
 "p(W' = ⋊.k.ɑ.n.f.l.ɪ.k.t.⋉.⋉) → 7.632333087715332e-06",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.æ.k.t.⋉.⋉) → 7.313382254003773e-06",
 "p(W' = ⋊.k.ɑ.n.s.ɛ.p.t.⋉.⋉) → 6.9986304968357715e-06",
 "p(W' = ⋊.k.ɑ.n.tʃ.ɪ.s.⋉.⋉) → 3.838134582818171e-06",
 "p(W' = ⋊.k.ɑ.n.f.ɪ.d.ɪ.n.s.⋉.⋉) → 3.3291579448602227e-06",
 "p(W' = ⋊.k.ɑ.n.s.ɚ.t.⋉.⋉) → 3.083158892404716e-06",
 "p(W' = ⋊.k.ɑ.n.t.ɛ.s.t.⋉.⋉) → 2.6089887049272665e-06",
 "p(W' = ⋊.k.ɑ.n.f.ɚ.ə.n.s.⋉.⋉) → 2.6089887049272665e-06",
 "p(W' = ⋊.k.ɑ.n.t.ɛ.k.s.t.⋉.⋉) → 2.381214689903069e-06",
 "p(W' = ⋊.k.ɑ.n.s.ɪ.n.t.ɹ.eɪ.t.⋉.⋉) → 1.9454157012935563e-06",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.v.ɚ.s.i.⋉.⋉) → 1.8407699291039008e-06",
 "p(W' = ⋊.k.ɑ.n.d.ʌ.k.t.⋉.⋉) → 1.7379196512084929e-06",
 "p(W' = ⋊.k.ɑ.n.k.ɹ.i.t.⋉.⋉) → 1.440555077125778e-06",
 "p(W' = ⋊.k.ɑ.n.s.ɪ.n.t.ɹ.eɪ.ʃ.ɪ.n.⋉.⋉) → 1.2521021286014346e-06",
 "p(W' = ⋊.k.ɑ.n.⋉.⋉) → 1.0160081485101533e-06",
 "p(W

["p(W' = ⋊.k.ɑ.n.v.ɚ.s.eɪ.ʃ.ɪ.n.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.03561657726952941",
 "p(W' = ⋊.k.ɑ.n.t.æ.k.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.030853005516955946",
 "p(W' = ⋊.k.ɑ.n.f.l.ɪ.k.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.025453184548080113",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.æ.k.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.027744037623243577",
 "p(W' = ⋊.k.ɑ.n.s.ɛ.p.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.0167610142827411",
 "p(W' = ⋊.k.ɑ.n.tʃ.ɪ.s.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.010051072041075713",
 "p(W' = ⋊.k.ɑ.n.f.ɪ.d.ɪ.n.s.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.012589405712290653",
 "p(W' = ⋊.k.ɑ.n.s.ɚ.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.008333866709453736",
 "p(W' = ⋊.k.ɑ.n.t.ɛ.s.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.006724179637401723",
 "p(W' = ⋊.k.ɑ.n.f.ɚ.ə.n.s.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.005262318304050252",
 "p(W' = ⋊.k.ɑ.n.t.ɛ.k.s.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.007382098377585577",
 "p(W' = ⋊.k.ɑ.n.s.ɪ.n.t.ɹ.eɪ.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.0054252249762038384",
 "p(W' = ⋊.k.ɑ.n.t.ɹ.ə.v.ɚ.s.i.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.005274340868871776",
 "p(W' = ⋊.k.ɑ.n.d.ʌ.k.t.⋉.⋉|r = '⋊.k.ɑ.n.t') → 0.

In [157]:
Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')

2773

In [159]:
'⋊.k.ɑ.n.t'
len( ds2t('⋊.k.ɑ.n.t') )
len( ds2t('⋊.k.ɑ.n.t') ) - 1
retrieve_CM_for('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', 4)

np.allclose( retrieve_CM_for('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', 4), 
             pY0k_X0ks[4][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')])

'⋊.k.ɑ.n.t'

5

4

array([[ 0.04390334,  0.01805178],
       [ 0.00700978,  0.01374583],
       [ 0.02545656,  0.00721312],
       [ 0.00700978,  0.00711758],
       [ 0.00682531,  0.00692774],
       [ 0.00700978,  0.00721312],
       [ 0.00682531,  0.0070233 ],
       [ 0.00700978,  0.00721312],
       [ 0.00682531,  0.00711758],
       [ 0.00700978,  0.00721312],
       [ 0.0051651 ,  0.00619171],
       [ 0.65246265,  0.00711758],
       [ 0.02545656,  0.00721312],
       [ 0.00700978,  0.00711758],
       [ 0.00700978,  0.00721312],
       [ 0.00700978,  0.01374583],
       [ 0.00700978,  0.00711758],
       [ 0.00682531,  0.00711758],
       [ 0.00700978,  0.00721312],
       [ 0.00700978,  0.00721312],
       [ 0.00700978,  0.00721312],
       [ 0.00682531,  0.00711758],
       [ 0.00664084,  0.00702074],
       [ 0.00682531,  0.0070233 ],
       [ 0.00700978,  0.01805178],
       [ 0.00682531,  0.00692774],
       [ 0.00700978,  0.00619171],
       [ 0.00700978,  0.59565208],
       [ 0.00700978,

True

# Definition of core calculation

In [160]:
from random import choice

In [161]:
random_wordform = choice(Ws_t)
random_wordform_length = len(ds2t(random_wordform))
random_wordform_idx = Ws_t.index(random_wordform)
random_wordform, random_wordform_idx, random_wordform_length

('⋊.d.ɪ.t.oʊ.⋉.⋉', 972, 7)

In [162]:
type(pW_C)
pW_C.shape
pW_C[:,0].shape

numpy.ndarray

(9172, 106295)

(9172,)

In [163]:
Cs_t[Cs_t.index('a couple of')]

'a couple of'

In [164]:
random_context = 'a couple of'

In [165]:
# pX0f = pW_C[:,Cs_t.index('a couple of')]
# pX0f_torch = torch.from_numpy(pX0f)
pW_C_torch = torch.from_numpy(pW_C)

In [166]:
def get_pW_t(c_idx):
    return pW_C_torch[:,c_idx]

In [167]:
# if r:
    CMsByLengthByWordformIndex = pY0k_X0ks
    CMsByLengthByWordformIndex_torch = [torch.from_numpy(each) for each in CMsByLengthByWordformIndex]
    exactCMsByLengthByWordformIndex = pY0f_X0fs
    exactCMsByLengthByWordformIndex_torch = [torch.tensor(each) for each in exactCMsByLengthByWordformIndex]
# else:
#     xCMsByLengthByWordformIndex_torch
#     raise Exception('Feature not implemented yet')
#     CMsByLengthByWordformIndex = pY0f_X0ks
#     CMsByLengthByWordformIndex_torch = [torch.from_numpy(each) for each in CMsByLengthByWordformIndex]
#     pass

In [168]:
random_wordform
ds2t(random_wordform)[1:-1]

'⋊.d.ɪ.t.oʊ.⋉.⋉'

('d', 'ɪ', 't', 'oʊ', '⋉')

In [169]:
random_wordform
ds2t(random_wordform)
random_wordform_idx
random_wordform_length
random_wordform_CM = CMsByLengthByWordformIndex_torch[random_wordform_length][random_wordform_idx]
'--'
retrieve_CM_for(random_wordform, len(ds2t(random_wordform)))
np.allclose(retrieve_CM_for(random_wordform, len(ds2t(random_wordform))),
            CMsByLengthByWordformIndex[random_wordform_length][random_wordform_idx])
'--'
CMsByLengthByWordformIndex_torch[random_wordform_length][random_wordform_idx].shape
random_wordform_CM

'⋊.d.ɪ.t.oʊ.⋉.⋉'

('⋊', 'd', 'ɪ', 't', 'oʊ', '⋉', '⋉')

972

7

'--'

array([[ 0.00700978,  0.00751919,  0.00705138,  0.00700978,  0.        ],
       [ 0.00700978,  0.00751919,  0.00705138,  0.06235012,  0.        ],
       [ 0.00700978,  0.00751919,  0.00705138,  0.00700978,  0.        ],
       [ 0.43128574,  0.0074196 ,  0.00695798,  0.00682531,  0.        ],
       [ 0.00682531,  0.0072217 ,  0.0067724 ,  0.00664084,  0.        ],
       [ 0.00700978,  0.09882666,  0.00705138,  0.00700978,  0.        ],
       [ 0.00682531,  0.00732132,  0.00686582,  0.00682531,  0.        ],
       [ 0.00700978,  0.00751919,  0.00705138,  0.00700978,  0.        ],
       [ 0.00682531,  0.0074196 ,  0.00695798,  0.00700978,  0.        ],
       [ 0.00700978,  0.03080986,  0.00705138,  0.00700978,  0.        ],
       [ 0.0051651 ,  0.00645444,  0.00605287,  0.00700978,  0.        ],
       [ 0.02527209,  0.0141393 ,  0.00695798,  0.00700978,  0.        ],
       [ 0.00700978,  0.00751919,  0.00705138,  0.00700978,  0.        ],
       [ 0.00700978,  0.0074196 ,  0.0

True

'--'

torch.Size([39, 5])

tensor([[0.0070, 0.0075, 0.0071, 0.0070, 0.0000],
        [0.0070, 0.0075, 0.0071, 0.0624, 0.0000],
        [0.0070, 0.0075, 0.0071, 0.0070, 0.0000],
        [0.4313, 0.0074, 0.0070, 0.0068, 0.0000],
        [0.0068, 0.0072, 0.0068, 0.0066, 0.0000],
        [0.0070, 0.0988, 0.0071, 0.0070, 0.0000],
        [0.0068, 0.0073, 0.0069, 0.0068, 0.0000],
        [0.0070, 0.0075, 0.0071, 0.0070, 0.0000],
        [0.0068, 0.0074, 0.0070, 0.0070, 0.0000],
        [0.0070, 0.0308, 0.0071, 0.0070, 0.0000],
        [0.0052, 0.0065, 0.0061, 0.0070, 0.0000],
        [0.0253, 0.0141, 0.0070, 0.0070, 0.0000],
        [0.0070, 0.0075, 0.0071, 0.0070, 0.0000],
        [0.0070, 0.0074, 0.0070, 0.0068, 0.0000],
        [0.0070, 0.0075, 0.0071, 0.0070, 0.0000],
        [0.0070, 0.0075, 0.0071, 0.5235, 0.0000],
        [0.0255, 0.0074, 0.0070, 0.0068, 0.0000],
        [0.0068, 0.0074, 0.0070, 0.0070, 0.0000],
        [0.2653, 0.0075, 0.7307, 0.0070, 0.0000],
        [0.0070, 0.0075, 0.0134, 0.0070, 0.0000],


## Channel sampler

In [170]:
# if not r:
    def depthSampler2a_t(CM, m=1):
        '''
        Given 
         - a sequence of channel distributions (i.e. a pytorch tensor)
           associated with some prefix x_0^i of the lexicon
         - a number of samples m
        this returns m samples from the channel distribution p(Y_0^i|x_0^i),
        where each 'sample' (i.e. channel string) is a sequence of one hot vectors.
        
        Let l = i+1:
        |depthSampler2a_t(|CM| = (|Y1|, l), m)| = (m, l, |Y1|)
        '''
        stack = torch.zeros((m, CM.shape[1], CM.shape[0]))
#         print(f'{stack.dtype}')
        for eachStack in torch.arange(m):
            for i in torch.arange(CM.shape[1]):
                stack[eachStack, i] = torch.distributions.Multinomial(1, CM[:,i]).sample()
        return stack
#         return torch.squeeze(stack)
# else:
#     def depthSampler2a_t(xCM, m=1):
#         stack = torch.zeros((m, xCM.shape[1], xCM.shape[0]))
#         for eachStack in torch.arange(m):
#             for i in torch.arange(xCM.shape[1]):
#                 stack[eachStack, i] = torch.distributions.Multinomial(1, xCM[:,i]).sample()
#         return stack

In [171]:
#FIXME copy what you need from the segmental sequence notebook to interpret these samples

In [172]:
random_wordform_CM.shape

torch.Size([39, 5])

In [173]:
random_wordform
sampled_channel_sequences = depthSampler2a_t(random_wordform_CM, m=2)
sampled_channel_sequences
channelSeqOHs2ds(sampled_channel_sequences.numpy()[0], False)
channelSeqOHs2ds(sampled_channel_sequences.numpy()[0], True)
sampled_channel_sequences.shape

'⋊.d.ɪ.t.oʊ.⋉.⋉'

tensor([[[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 1.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

'd.eɪ.s.ʌ.⋉'

'⋊.d.eɪ.s.ʌ.⋉'

torch.Size([2, 5, 39])

In [174]:
random_wordform
sampled_channel_sequences2 = depthSampler2a_t(random_wordform_CM, m=100)
[channelSeqOHs2ds(each.numpy(), True) for each in sampled_channel_sequences2]

'⋊.d.ɪ.t.oʊ.⋉.⋉'

['⋊.ð.ɪ.t.oʊ.⋉',
 '⋊.d.ɪ.eɪ.oʊ.⋉',
 '⋊.t.eɪ.t.ɚ.⋉',
 '⋊.k.ɪ.tʃ.oʊ.⋉',
 '⋊.θ.eɪ.t.ɚ.⋉',
 '⋊.ɪ.ɛ.t.ɚ.⋉',
 '⋊.d.ɪ.t.ɚ.⋉',
 '⋊.t.ɛ.t.oʊ.⋉',
 '⋊.t.ɪ.t.oʊ.⋉',
 '⋊.d.ɪ.t.d.⋉',
 '⋊.j.ɪ.t.oʊ.⋉',
 '⋊.d.ɪ.oʊ.ɹ.⋉',
 '⋊.d.ɪ.t.oʊ.⋉',
 '⋊.d.ð.t.oʊ.⋉',
 '⋊.t.k.t.ɚ.⋉',
 '⋊.d.ɪ.aɪ.s.⋉',
 '⋊.d.ɪ.t.d.⋉',
 '⋊.d.eɪ.t.oʊ.⋉',
 '⋊.t.ɪ.t.oʊ.⋉',
 '⋊.d.aɪ.t.oʊ.⋉',
 '⋊.l.eɪ.t.oʊ.⋉',
 '⋊.k.i.t.ɚ.⋉',
 '⋊.d.ɪ.t.oʊ.⋉',
 '⋊.d.ɪ.t.oʊ.⋉',
 '⋊.f.eɪ.t.oʊ.⋉',
 '⋊.d.ɹ.t.oʊ.⋉',
 '⋊.oʊ.v.ɪ.oʊ.⋉',
 '⋊.t.ɪ.t.ʒ.⋉',
 '⋊.d.ɹ.z.oʊ.⋉',
 '⋊.d.æ.h.n.⋉',
 '⋊.ʒ.ɪ.t.g.⋉',
 '⋊.t.ɪ.tʃ.ɛ.⋉',
 '⋊.n.j.t.ɚ.⋉',
 '⋊.d.ɪ.t.oʊ.⋉',
 '⋊.d.ʊ.t.dʒ.⋉',
 '⋊.n.ɪ.ɪ.oʊ.⋉',
 '⋊.t.ɪ.t.u.⋉',
 '⋊.l.ɪ.t.ɔɪ.⋉',
 '⋊.ʊ.ɪ.t.ʌ.⋉',
 '⋊.p.ɪ.ʊ.ɚ.⋉',
 '⋊.d.ʃ.t.ɚ.⋉',
 '⋊.d.ɪ.t.oʊ.⋉',
 '⋊.d.i.t.oʊ.⋉',
 '⋊.ʊ.eɪ.t.oʊ.⋉',
 '⋊.t.ɪ.t.oʊ.⋉',
 '⋊.d.ɪ.t.ɚ.⋉',
 '⋊.t.i.t.u.⋉',
 '⋊.t.ɔɪ.t.oʊ.⋉',
 '⋊.s.ɪ.t.oʊ.⋉',
 '⋊.t.ɪ.i.ɛ.⋉',
 '⋊.ɪ.ɪ.t.ɔɪ.⋉',
 '⋊.t.ɪ.t.oʊ.⋉',
 '⋊.p.ɪ.t.f.⋉',
 '⋊.d.aɪ.t.aʊ.⋉',
 '⋊.t.ɪ.t.oʊ.⋉',
 '⋊.d.ɪ.t.oʊ.⋉',
 '⋊.t.ɪ.t.tʃ.⋉',
 '⋊.z.ɪ.t.ʊ.⋉',
 '⋊.ɑ.ɪ.p.oʊ.⋉',
 '⋊.d.

In [175]:
random_wordform
sampled_channel_sequences3 = [sample_pY0iX0k(random_wordform) for each in range(50)]; sampled_channel_sequences3

'⋊.d.ɪ.t.oʊ.⋉.⋉'

['⋊.ŋ.ɪ.t.oʊ.⋉.⋉',
 '⋊.d.ɪ.t.oʊ.⋉.⋉',
 '⋊.i.ɪ.θ.ɚ.⋉.⋉',
 '⋊.t.h.t.oʊ.⋉.⋉',
 '⋊.p.ɪ.t.oʊ.⋉.⋉',
 '⋊.t.aʊ.eɪ.ɚ.⋉.⋉',
 '⋊.θ.p.tʃ.oʊ.⋉.⋉',
 '⋊.i.ɪ.t.aʊ.⋉.⋉',
 '⋊.d.ɪ.t.b.⋉.⋉',
 '⋊.t.u.t.ɑ.⋉.⋉',
 '⋊.t.ɪ.ʒ.h.⋉.⋉',
 '⋊.ʊ.ɛ.t.g.⋉.⋉',
 '⋊.d.ɪ.t.oʊ.⋉.⋉',
 '⋊.t.ɪ.t.oʊ.⋉.⋉',
 '⋊.d.ʊ.j.z.⋉.⋉',
 '⋊.k.ɪ.t.oʊ.⋉.⋉',
 '⋊.t.ɪ.t.oʊ.⋉.⋉',
 '⋊.p.ɪ.l.v.⋉.⋉',
 '⋊.t.p.l.ð.⋉.⋉',
 '⋊.p.eɪ.t.oʊ.⋉.⋉',
 '⋊.d.ɪ.θ.ʌ.⋉.⋉',
 '⋊.d.f.t.oʊ.⋉.⋉',
 '⋊.t.ɪ.t.ʊ.⋉.⋉',
 '⋊.d.ɪ.t.oʊ.⋉.⋉',
 '⋊.d.ɪ.t.oʊ.⋉.⋉',
 '⋊.t.ɪ.t.ʌ.⋉.⋉',
 '⋊.t.ɪ.t.aʊ.⋉.⋉',
 '⋊.d.ɪ.t.oʊ.⋉.⋉',
 '⋊.d.dʒ.t.oʊ.⋉.⋉',
 '⋊.d.ɪ.t.oʊ.⋉.⋉',
 '⋊.d.l.t.oʊ.⋉.⋉',
 '⋊.t.ɪ.t.oʊ.⋉.⋉',
 '⋊.ʊ.l.t.oʊ.⋉.⋉',
 '⋊.t.ɪ.t.oʊ.⋉.⋉',
 '⋊.t.eɪ.ð.ɪ.⋉.⋉',
 '⋊.t.ɪ.tʃ.l.⋉.⋉',
 '⋊.d.ɪ.t.aʊ.⋉.⋉',
 '⋊.t.ɪ.tʃ.aʊ.⋉.⋉',
 '⋊.t.ɪ.eɪ.θ.⋉.⋉',
 '⋊.d.tʃ.t.u.⋉.⋉',
 '⋊.ʌ.ɪ.u.ɹ.⋉.⋉',
 '⋊.d.ɪ.t.aʊ.⋉.⋉',
 '⋊.t.ɪ.t.oʊ.⋉.⋉',
 '⋊.m.ɪ.t.u.⋉.⋉',
 '⋊.d.ɪ.t.oʊ.⋉.⋉',
 '⋊.d.z.t.ɚ.⋉.⋉',
 '⋊.t.ɑ.t.oʊ.⋉.⋉',
 '⋊.d.ɪ.t.oʊ.⋉.⋉',
 '⋊.d.v.t.aʊ.⋉.⋉',
 '⋊.ð.æ.t.oʊ.⋉.⋉']

In [176]:
len(ds2t('⋊.ɔɪ.ŋ.dʒ.l.aɪ.h'))
y0kOHmap('⋊.ɔɪ.ŋ.dʒ.l.aɪ.h')
y0kOHmap('⋊.ɔɪ.ŋ.dʒ.l.aɪ.h').shape
np.array(list(map(y0kOHmap, sampled_channel_sequences3))).shape
np.array(list(map(y0kOHmap, [sample_pY0iX0k(random_wordform) for each in range(50)])))

7

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,

(6, 39)

(50, 6, 39)

array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  1., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.],
        [ 0.,  0.,  0., ...,  0.,  0.,  1.]],

       ..., 
       [[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0

In [177]:
torch.tensor(sample_pY0iX0k(random_wordform, debug=True)[1]) == random_wordform_CM

tensor([[1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 1, 1]], dtype=torch.uint8)

In [178]:
# channelSeqOHs2ds(np.array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
#           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
#           0., 0., 0., 0.]]))

## Sample-parallel posterior estimator

In [179]:
# if not r:
#     def pXhat0fX0i_pxt(xhat0f_idx, x0k_CM=None, m = 50, x0k=None):
    def pXhat0fX0i_pxt(xhat0f_idx, x0k_CM, c_idx, m = 50):
        '''
        If ⋊ x_0 x_1 ... x_i x_k denotes
        the produced prefix r of the speaker's intended wordform w,
         where
           x_i = the last fully produced segment
           k = i+1
           x_k = the next segment
           |⋊ x_0 x_1 ... x_i x_k| = l + 2
           l = i+1 = the length of the produced prefix (not incl. 
                     the left word edge or the next segment)
        
        then
        
        x0k_CM :: (|Y1|, l) is the associated l-length stack of size |Y1| vectors
         where x0k_CM[:,j] is the distribution p(Y_j|X0^k has been produced)
           
        xhat0f_idx is the index of a wordform w' s.t. 
        
        pXhat0fX0i_pxt(xhat0f_idx, x0k_CM) ≈ p(w'|r)
        '''
        shape_info = False
        pW = get_pW_t(c_idx)
#         if x0k_CM is None and x0k is None:
#             raise Exception('Must specify one of x0k_CM or x0k.')
        
        # Computation proceeds in two steps:
        # calculate the denominator p(y0i) for m = 50 sampled y0i's
        # calculate the numerator p(y0i|w')p(w') for m = 50 sampled y0i's
        
        
    #     xhat0f_idx = Wmap[xhat0f]

    #     l = len(ds2t(x0i))
    #     x0k_CM = CMsByPrefixIndex[prefixMap[x0i]]

    #     my_Q_l = CMsByLengthByWordformIndex[l - 2]

        #Collect m samples from p(Y0i|X0k) = p(Y0i|r), 
        # where each sample is an l-length stack of one-hot vectors
        # where each one-hot vector corresponds to a channel symbol:
        Y_prime = depthSampler2a_t(x0k_CM, m)#.float() #:: (m, l, |Y1|)
#         if x0k_CM is not None:
#             Y_prime = depthSampler2a_t(x0k_CM, m)#.float() #:: (m, l, |Y1|)
#             if x0k is not None:
# #                 print('foo')
#                 print(type(x0k_CM))
#                 print(x0k_CM.shape)
#                 generated_CM = sample_pY0iX0k(x0k, debug=True)[1]
#                 print(type(generated_CM))
#                 print(generated_CM.shape)
#                 generated_CM_t = torch.tensor(generated_CM)
#                 assert torch.allclose(x0k_CM, torch.tensor(sample_pY0iX0k(x0k, debug=True)[1]))
# #                 print('bar')
#         else:
#             Y_prime = torch.tensor(list(map(y0kOHmap, [sample_pY0iX0k(x0k) for each in range(m)])))
#         print('baz')
        l = Y_prime.shape[1]# + 1
        if shape_info:
            print(f'|Y_prime| = {Y_prime.shape} = (m, l, |Y1|)')

        #Grab ALL the relevant-prefix-length channel matrices 
        # for every segmental wordform in the lexicon:
#         my_Q_l = CMsByLengthByWordformIndex_torch[l - 3]#.float()
#         my_Q_l = CMsByLengthByWordformIndex_torch[l - 1]#.float() #:: (|W|, |Y1|, l)
        my_Q_l = CMsByLengthByWordformIndex_torch[l+2]#.float() #:: (|W|, |Y1|, l)
        if shape_info:
            print(f'|Q_l| = {my_Q_l.shape} = (|W|=n, |Y1|, l)')
    
#         print(f'{Y_prime.dtype}, {my_Q_l.dtype}')
    
        # NORMALIZATION
        V_prime = torch.einsum('mli,kil->mkl', Y_prime, my_Q_l)  # :: (m,n,l)
#         print(f"|V'| = {V_prime.shape} = (m, n, l)")
        M_prime = torch.prod(V_prime, 2) # :: (m,n)
#         print(f"|M'| = {M_prime.shape} = (m, n)")
#         N_prime = torch.matmul(M_prime, pX0f_torch) # :: (m, 1) <- prior probabilities of each of the m sampled channel prefixes
        N_prime = torch.matmul(M_prime, pW)
#         print(f"|N'| = {N_prime.shape} = (m, )")
        Z_prime = 1.0 / N_prime # :: (m, 1)
#         print(f"|Z'| = {Z_prime.shape} = (m, )")
        if shape_info:
            print(f"|V'| = {V_prime.shape} = (m, n, l)")
            print(f"|M'| = {M_prime.shape} = (m, n)")
            print(f"|N'| = {N_prime.shape} = (m, )")
            print(f"|Z'| = {Z_prime.shape} = (m, )")

        # NUMERATOR
        L_w = my_Q_l[xhat0f_idx]#.float()
        V_prime_w = torch.einsum('mij,ji->mi',Y_prime, L_w)
#         print(f"|V'_w| = {V_prime_w.shape} = (m, l)")
        O_w = torch.prod(V_prime_w, 1) # :: (m,1) likelihoods of each of the m sampled channel prefixes
#         print(f"|O_w| = {O_w.shape} = (m, )")
#         U_w = pX0f_torch[xhat0f_idx] * O_w ## :: (m,1) joint probabilities of xhat0f with each of the m sampled channel prefixes
        U_w = pW[xhat0f_idx] * O_w ## :: (m,1) joint probabilities of xhat0f with each of the m sampled channel prefixes
#         print(f"|U_w| = {U_w.shape} = (m, )")
        if shape_info:
            print(f"|V'_w| = {V_prime_w.shape} = (m, l)")
            print(f"|O_w| = {O_w.shape} = (m, )")
            print(f"|U_w| = {U_w.shape} = (m, )")
        
        
        if shape_info:
            E = torch.dot(Z_prime, U_w) / m
            print(f"|E| = scalar: type(E) = {type(E)}")
        return torch.dot(Z_prime, U_w) / m

    #     return torch.dot( 1.0 / torch.matmul(torch.prod(torch.einsum('mli,kil->mkl', Y_prime, my_Q_l), 2), pX0f_torch) , 
    #        pX0f_torch[xhat0f_idx] * torch.prod(torch.einsum('mij,ji->mi',Y_prime, L_w), 1) ) / m
# else:
#     def pXhat0fX0k_pxt(xhat0f_idx, x0i_xCM, m = 50):
#     #     xhat0f_idx = Wmap[xhat0f]

#     #     l = len(ds2t(x0i))
#     #     x0i_xCM = xCMsByPrefixIndex[prefixMap[x0i]]

#     #     my_Q_l = xCMsByLengthByWordformIndex[l - 2]

#         Y_prime = depthSampler2a_t(x0i_xCM, m)#.float()
#         l = Y_prime.shape[1] + 1

#         my_Q_l = xCMsByLengthByWordformIndex_torch[l - 3]#.float()

#         # NORMALIZATION
#         V_prime = torch.einsum('mli,kil->mkl', Y_prime, my_Q_l)  # :: (m,n,l)
#         M_prime = torch.prod(V_prime, 2) # :: (m,n)
#         N_prime = torch.matmul(M_prime, pX0f_torch) # :: (m, 1) <- prior probabilities of each of the m sampled channel prefixes
#         Z_prime = 1.0 / N_prime # :: (m, 1)

#         # NUMERATOR
#         L_w = my_Q_l[xhat0f_idx]#.float()
#         V_prime_w = torch.einsum('mij,ji->mi',Y_prime, L_w)
#         O_w = torch.prod(V_prime_w, 1) # :: (m,1) likelihoods of each of the m sampled channel prefixes
#         U_w = pX0f_torch[xhat0f_idx] * O_w ## :: (m,1) joint probabilities of xhat0f with each of the m sampled channel prefixes

#         return torch.dot(Z_prime, U_w) / m

#     #     return torch.dot( 1.0 / torch.matmul(torch.prod(torch.einsum('mli,kil->mkl', Y_prime, my_Q_l), 2), pX0f_torch) , 
#     #        pX0f_torch[xhat0f_idx] * torch.prod(torch.einsum('mij,ji->mi',Y_prime, L_w), 1) ) / m

In [180]:
def pXhat0fX0i_pxtn(xhat0f_idx, x0k_CM, c_idx, m = 50):
    return pXhat0fX0i_pxt(xhat0f_idx, x0k_CM, c_idx, m = 50).numpy()

In [181]:
def prefixTox0kCM(r):
    source_wf = list(wordsWithPrefix(r, Ws))[0]
    source_wf_idx = Ws_t.index(source_wf)
    
    r_t = ds2t(r)
    total_x0k_length = len(r_t)# includes left edge symbol + upcoming segment
#     prefix_length_noWE = total_x0k_length - 1 #includes upcoming segment
#     prefix_length_produced_so_far = prefix_length_noWE - 1
#     offset = 1
#     x0k_cm = CMsByLengthByWordformIndex_torch[prefix_length_produced_so_far - offset][source_wf_idx]
    x0k_cm = CMsByLengthByWordformIndex_torch[total_x0k_length][source_wf_idx]
    return x0k_cm

In [182]:
CMsByLengthByWordformIndex_torch[3].shape
CMsByLengthByWordformIndex_torch[4].shape
len(CMsByLengthByWordformIndex_torch)

torch.Size([0, 0])

torch.Size([9172, 39, 2])

20

In [183]:
len(Ws_t)

9172

In [184]:
def wordformTox0kCM(w=None, w_idx=None):
#     if w is None and w_idx is None:
#         raise Exception('At least one argument must be specified.')
    if w is None:
        w = Ws_t[w_idx]
        w_t = ds2t(w)
    else:
        w_t = ds2t(w)
        w_idx = Ws_t.index(w)
    
    total_x0f_length = len(w_t) #includes both edge symbols
#     word_length_noLE = total_x0f_length - 1 #still includes right edge symbol = "upcoming segment"
#     word_length_noRE = word_length_noLE - 1
#     offset = 2
#     assert word_length_produced_so_far - offset >= 0 , f"{word_length_produced_so_far} - {offset} < 0; w = {w}"
#     assert word_length_produced_so_far - offset < len(CMsByLengthByWordformIndex_torch), f"{word_length_produced_so_far} - {offset} >= {len(CMsByLengthByWordformIndex_torch)}; w = {w}"
#     assert w_idx in range(CMsByLengthByWordformIndex_torch[word_length_produced_so_far - offset].shape[0]), f"{w_idx}; w = {w}"
#     x0f_cm = CMsByLengthByWordformIndex_torch[word_length_produced_so_far - offset][w_idx]
    x0f_cm = CMsByLengthByWordformIndex_torch[total_x0f_length][w_idx]
    return x0f_cm

# def wordformTox0fCM(w=None, w_idx=None):
# #     if w is None and w_idx is None:
# #         raise Exception('At least one argument must be specified.')
#     if w is None:
#         w = Ws_t[w_idx]
#         w_t = ds2t(w)
#     else:
#         w_t = ds2t(w)
#         w_idx = Ws_t.index(w)
    
#     total_x0f_length = len(w_t) #includes both edge symbols
# #     word_length_noLE = total_x0f_length - 1 #still includes right edge symbol = "upcoming segment"
# #     word_length_noRE = word_length_noLE - 1
# #     offset = 2
# #     assert word_length_produced_so_far - offset >= 0 , f"{word_length_produced_so_far} - {offset} < 0; w = {w}"
# #     assert word_length_produced_so_far - offset < len(CMsByLengthByWordformIndex_torch), f"{word_length_produced_so_far} - {offset} >= {len(CMsByLengthByWordformIndex_torch)}; w = {w}"
# #     assert w_idx in range(CMsByLengthByWordformIndex_torch[word_length_produced_so_far - offset].shape[0]), f"{w_idx}; w = {w}"
# #     x0f_cm = CMsByLengthByWordformIndex_torch[word_length_produced_so_far - offset][w_idx]
#     x0f_cm = exactCMsByLengthByWordformIndex_torch[total_x0f_length][w_idx]
#     return x0f_cm

In [185]:
len(ds2t('⋊.m.ɪ.n.ɪ.s.t'))
len(dsTo3factors('⋊.m.ɪ.n.ɪ.s.t'))
prefixTox0kCM('⋊.m.ɪ.n.ɪ.s.t').shape

len(ds2t('⋊.m.ɪ.n.ɪ.s.t.ɚ'))
len(dsTo3factors('⋊.m.ɪ.n.ɪ.s.t.ɚ'))
prefixTox0kCM('⋊.m.ɪ.n.ɪ.s.t.ɚ').shape

len(ds2t('⋊.m.ɪ.n.ɪ.s.t.ɚ.⋉'))
len(dsTo3factors('⋊.m.ɪ.n.ɪ.s.t.ɚ.⋉'))
prefixTox0kCM('⋊.m.ɪ.n.ɪ.s.t.ɚ.⋉').shape

len(ds2t('⋊.m.ɪ.n.ɪ.s.t.ɚ.⋉.⋉'))
len(dsTo3factors('⋊.m.ɪ.n.ɪ.s.t.ɚ.⋉.⋉'))
prefixTox0kCM('⋊.m.ɪ.n.ɪ.s.t.ɚ.⋉.⋉').shape

7

5

torch.Size([39, 5])

8

6

...

7

torch.Size([39, 7])

10

8

torch.Size([39, 8])

In [186]:
random_source_prefix
len(ds2t(random_source_prefix)) - 2
prefixTox0kCM(random_source_prefix).shape
prefixTox0kCM(random_source_prefix)

'⋊.ɪ.d.ɪ.ʃ.ɪ'

4

torch.Size([39, 4])

tensor([[0.0070, 0.0078, 0.0078, 0.0071],
        [0.0070, 0.0078, 0.0078, 0.0071],
        [0.0070, 0.0233, 0.0078, 0.0071],
        [0.0070, 0.5818, 0.0077, 0.0070],
        [0.0068, 0.0075, 0.0075, 0.0069],
        [0.0808, 0.0078, 0.0344, 0.0071],
        [0.0068, 0.0076, 0.0076, 0.0069],
        [0.0070, 0.0078, 0.0078, 0.0071],
        [0.0068, 0.0077, 0.0077, 0.0070],
        [0.0070, 0.0078, 0.0320, 0.0136],
        [0.0052, 0.0067, 0.0067, 0.0061],
        [0.0068, 0.0148, 0.0147, 0.0070],
        [0.0070, 0.0078, 0.0078, 0.0071],
        [0.0070, 0.0077, 0.0077, 0.0070],
        [0.0070, 0.0149, 0.0078, 0.0071],
        [0.0070, 0.0078, 0.0078, 0.0071],
        [0.0255, 0.0147, 0.0077, 0.0176],
        [0.0068, 0.0077, 0.0077, 0.0135],
        [0.0070, 0.0914, 0.0078, 0.0071],
        [0.0070, 0.0078, 0.0078, 0.0071],
        [0.0070, 0.0078, 0.0078, 0.0071],
        [0.0068, 0.0077, 0.0077, 0.0070],
        [0.0066, 0.0076, 0.0076, 0.0069],
        [0.0068, 0.0076, 0.0076, 0

In [187]:
# for convenience
def pXhat0fX0i_pxtn_conv(w, r, c, m = 50):
    w_idx = Ws_t.index(w)

#     r_idx = Ps_t.index(r)
    source_wf = list(wordsWithPrefix(r, Ws))[0]
    source_wf_idx = Ws_t.index(source_wf)
    
    total_x0k_length = len(ds2t(r))# includes left edge symbol + upcoming segment
#     prefix_length_noWE = total_x0k_length - 1 #includes upcoming segment
#     prefix_length_produced_so_far = prefix_length_noWE - 1
#     offset = 1
#     x0k_cm = CMsByLengthByWordformIndex_torch[prefix_length_produced_so_far - offset][source_wf_idx]
    x0k_cm = CMsByLengthByWordformIndex_torch[total_x0k_length][source_wf_idx]
    
    c_idx = Cs_t.index(c)
    
    return pXhat0fX0i_pxtn(w_idx, x0k_cm, c_idx, m=m)

In [188]:
random_source_prefix
len(ds2t(random_source_prefix)) - 2
CMsByLengthByWordformIndex_torch[len(ds2t(random_source_prefix))][Ws_t.index(list(wordsWithPrefix(random_source_prefix, Ws))[0])].shape
CMsByLengthByWordformIndex_torch[len(ds2t(random_source_prefix))][Ws_t.index(list(wordsWithPrefix(random_source_prefix, Ws))[0])]

'⋊.ɪ.d.ɪ.ʃ.ɪ'

4

torch.Size([39, 4])

tensor([[0.0070, 0.0078, 0.0078, 0.0071],
        [0.0070, 0.0078, 0.0078, 0.0071],
        [0.0070, 0.0233, 0.0078, 0.0071],
        [0.0070, 0.5818, 0.0077, 0.0070],
        [0.0068, 0.0075, 0.0075, 0.0069],
        [0.0808, 0.0078, 0.0344, 0.0071],
        [0.0068, 0.0076, 0.0076, 0.0069],
        [0.0070, 0.0078, 0.0078, 0.0071],
        [0.0068, 0.0077, 0.0077, 0.0070],
        [0.0070, 0.0078, 0.0320, 0.0136],
        [0.0052, 0.0067, 0.0067, 0.0061],
        [0.0068, 0.0148, 0.0147, 0.0070],
        [0.0070, 0.0078, 0.0078, 0.0071],
        [0.0070, 0.0077, 0.0077, 0.0070],
        [0.0070, 0.0149, 0.0078, 0.0071],
        [0.0070, 0.0078, 0.0078, 0.0071],
        [0.0255, 0.0147, 0.0077, 0.0176],
        [0.0068, 0.0077, 0.0077, 0.0135],
        [0.0070, 0.0914, 0.0078, 0.0071],
        [0.0070, 0.0078, 0.0078, 0.0071],
        [0.0070, 0.0078, 0.0078, 0.0071],
        [0.0068, 0.0077, 0.0077, 0.0070],
        [0.0066, 0.0076, 0.0076, 0.0069],
        [0.0068, 0.0076, 0.0076, 0

In [189]:
(random_source_wordform, random_source_prefix, random_context)
pXhat0fX0i_pxtn_conv(random_source_wordform, random_source_prefix, random_context)

('⋊.tʃ.ɛ.k.ɚ.⋉.⋉', '⋊.ɪ.d.ɪ.ʃ.ɪ', 'a couple of')

array(2.618461217875298e-09)

## Testing

In [190]:
# random_wordform
# random_wordform_length
# retrieve_CM_for(random_wordform, random_wordform_length-2).shape
# np.allclose(retrieve_CM_for(random_wordform, random_wordform_length-2),
#             random_wordform_CM.numpy())
# # random_wordform_CM = CMsByLengthByWordformIndex_torch[random_wordform_length-3][random_wordform_idx]
# random_wordform_idx
# ' '
# Y_PRIME = depthSampler2a_t(random_wordform_CM, m=2); Y_PRIME.shape
# L = Y_PRIME.shape[1]; L
# pXhat0fX0i_pxt(random_wordform_idx, random_wordform_CM)
# # CMsByLengthByWordformIndex_torch[L - 3].shape # <<< this should either be (|Ws| x |Y1s| x ?)

In [191]:
# len(Ws_t)

In [192]:
# random_context = choice(Cs_t); random_context
random_context = 'a couple of'
random_context_idx = Cs_t.index(random_context); random_context_idx

1753

In [193]:
random_wordform, random_wordform, random_context
pXhat0fX0i_pxt(random_wordform_idx, 
               random_wordform_CM,
               random_context_idx)

('⋊.d.ɪ.t.oʊ.⋉.⋉', '⋊.d.ɪ.t.oʊ.⋉.⋉', 'a couple of')

tensor(0.0031)

In [194]:
Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')

'⋊.k.ɑ.n.t'
len( ds2t('⋊.k.ɑ.n.t') )
len( ds2t('⋊.k.ɑ.n.t') ) - 2
retrieve_CM_for('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', 5)

np.allclose( retrieve_CM_for('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', 5), 
             CMsByLengthByWordformIndex[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')])

2773

'⋊.k.ɑ.n.t'

5

3

array([[ 0.04390334,  0.01805178,  0.00911065],
       [ 0.00700978,  0.01374583,  0.00911065],
       [ 0.02545656,  0.00721312,  0.00911065],
       [ 0.00700978,  0.00711758,  0.00898997],
       [ 0.00682531,  0.00692774,  0.01700733],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.00682531,  0.0070233 ,  0.0088709 ],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.00682531,  0.00711758,  0.00898997],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.0051651 ,  0.00619171,  0.00782054],
       [ 0.65246265,  0.00711758,  0.00898997],
       [ 0.02545656,  0.00721312,  0.00911065],
       [ 0.00700978,  0.00711758,  0.00898997],
       [ 0.00700978,  0.00721312,  0.6450247 ],
       [ 0.00700978,  0.01374583,  0.00911065],
       [ 0.00700978,  0.00711758,  0.00898997],
       [ 0.00682531,  0.00711758,  0.00898997],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.00700978,  0.00721312,  0.00911065],
       [ 0.00700978,  0.00721312,  0.009

True

In [197]:
pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'),
               CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')],
               Cs_t.index('a couple of'),
               m = 50)

tensor(0.0023)

In [198]:
# sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 10) for each in range(10)])/10
# sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 50) for each in range(10)])/10
# sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 100) for each in range(10)])/10
# phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 250)
# phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 500)
# phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 1000)
# phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 2500)
# phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 5000)
# phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉', '⋊.k.ɑ.n.t', 10000)
f"m = 10    → {sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 10) for each in range(10)])/10}"
f"m = 50    → {sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 50) for each in range(10)])/10}"
f"m = 100   → {sum([phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 100) for each in range(10)])/10}"
f"m = 250   → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 250)}"
f"m = 500   → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 500)}"
f"m = 1000  → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 1000)}"
f"m = 2500  → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 2500)}"
f"m = 5000  → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 5000)}"
f"m = 10000 → {phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉', '⋊.k.ɑ.n.t', random_context, 10000)}"

'm = 10    → 0.0029988860836367218'

'm = 50    → 0.002990670533478189'

'm = 100   → 0.003463228338473098'

'm = 250   → 0.003043302135665002'

'm = 500   → 0.003520302056559275'

'm = 1000  → 0.0033861026543259257'

'm = 2500  → 0.003253028203464568'

'm = 5000  → 0.003273181596255034'

'm = 10000 → 0.0033026730912458766'

In [199]:
#f"m = 50   → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), x0k = '⋊.k.ɑ.n.t', m = 50)}"
#f"m = 100  → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), x0k = '⋊.k.ɑ.n.t', m = 100)}"
#f"m = 200  → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), x0k = '⋊.k.ɑ.n.t', m = 200)}"
#f"m = 500  → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), x0k = '⋊.k.ɑ.n.t', m = 500)}"
#f"m = 1000 → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), x0k = '⋊.k.ɑ.n.t', m = 1000)}"
#f"m = 2000 → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), x0k = '⋊.k.ɑ.n.t', m = 2000)}"
#f"m = 5000 → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'), x0k = '⋊.k.ɑ.n.t', m = 5000)}"

In [201]:
# pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
#               CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
#               m = 50)
# pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
#               CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
#               m = 100)
# pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
#               CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
#               m = 200)
# pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
#               CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
#               m = 500)
# pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
#               CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
#               m = 1000)
# pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
#               CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
#               m = 2000)
# pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉'),
#               CMsByLengthByWordformIndex_torch[4-1][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉')],
#               m = 5000)
f"m = 50   → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], random_context_idx, m = 50)}"
f"m = 100  → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], random_context_idx, m = 100)}"
f"m = 200  → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], random_context_idx, m = 200)}"
f"m = 500  → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], random_context_idx, m = 500)}"
f"m = 1000 → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], random_context_idx, m = 1000)}"
f"m = 2000 → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], random_context_idx, m = 2000)}"
f"m = 5000 → {pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'), CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')], random_context_idx, m = 5000)}"

'm = 50   → 0.003684943760778252'

'm = 100  → 0.0036732943162272164'

'm = 200  → 0.0032676742692132952'

'm = 500  → 0.0030396006632736647'

'm = 1000 → 0.0029672361353951654'

'm = 2000 → 0.0031396328876238115'

'm = 5000 → 0.0032260666870033154'

In [202]:
%%timeit

pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'),
              CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')],
              Cs_t.index('a couple of'), 
              m = 50)

38.7 ms ± 675 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [203]:
%%timeit

pXhat0fX0i_pxtn(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'),
                CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')],
                Cs_t.index('a couple of'),
                m = 50)

39 ms ± 2.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [204]:
%%timeit

pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'),
               CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')],
               Cs_t.index('a couple of'),
               m = 200)

148 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [205]:
Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')
Ps_t.index('⋊.k.ɑ.n.t')

2773

12129

## Exact/full wordform-length variant functions

These variant versions of the functions from section 5.2 are used for Case 3 calculations.

In [206]:
def pXhat0fX0f_pxt(xhat0f_idx, x0f_CM, c_idx, m = 50):
    '''
    If ⋊ x_0 x_1 ... x_i x_k denotes
    the produced prefix r of the speaker's intended wordform w,
     where
       x_i = the last fully produced segment
       k = i+1
       x_k = the next segment
       |⋊ x_0 x_1 ... x_i x_k| = l + 2
       l = i+1 = the length of the produced prefix (not incl. 
                 the left word edge or the next segment)

    then

    x0f_CM :: (|Y1|, l) is the associated l-length stack of size |Y1| vectors
     where x0f_CM[:,j] is the distribution p(Y_j|X0^f has been produced)

    xhat0f_idx is the index of a wordform w' s.t. 
        pXhat0fX0f_pxt(xhat0f_idx, x0f_CM) ≈ p(w'|w)
    '''
    shape_info = False
    pW = get_pW_t(c_idx)
#     if x0k_CM is None and x0k is None:
#         raise Exception('Must specify one of x0k_CM or x0k.')

    # Computation proceeds in two steps:
    # calculate the denominator p(y0i) for m = 50 sampled y0i's
    # calculate the numerator p(y0i|w')p(w') for m = 50 sampled y0i's


#     xhat0f_idx = Wmap[xhat0f]

#     l = len(ds2t(x0i))
#     x0k_CM = CMsByPrefixIndex[prefixMap[x0i]]

#     my_Q_l = CMsByLengthByWordformIndex[l - 2]

    #Collect m samples from p(Y0i|X0k) = p(Y0i|r), 
    # where each sample is an l-length stack of one-hot vectors
    # where each one-hot vector corresponds to a channel symbol:
    Y_prime = depthSampler2a_t(x0f_CM, m)#.float() #:: (m, l, |Y1|)
#         if x0k_CM is not None:
#             Y_prime = depthSampler2a_t(x0k_CM, m)#.float() #:: (m, l, |Y1|)
#             if x0k is not None:
# #                 print('foo')
#                 print(type(x0k_CM))
#                 print(x0k_CM.shape)
#                 generated_CM = sample_pY0iX0k(x0k, debug=True)[1]
#                 print(type(generated_CM))
#                 print(generated_CM.shape)
#                 generated_CM_t = torch.tensor(generated_CM)
#                 assert torch.allclose(x0k_CM, torch.tensor(sample_pY0iX0k(x0k, debug=True)[1]))
# #                 print('bar')
#         else:
#             Y_prime = torch.tensor(list(map(y0kOHmap, [sample_pY0iX0k(x0k) for each in range(m)])))
#         print('baz')
    l = Y_prime.shape[1]# + 1
    if shape_info:
        print(f'|Y_prime| = {Y_prime.shape} = (m, l, |Y1|)')

    #Grab ALL the relevant-prefix-length channel matrices 
    # for every segmental wordform in the lexicon:
#         my_Q_l = CMsByLengthByWordformIndex_torch[l - 3]#.float()
#         my_Q_l = CMsByLengthByWordformIndex_torch[l - 1]#.float() #:: (|W|, |Y1|, l)
    my_Q_l = exactCMsByLengthByWordformIndex_torch[l+2]#.float() #:: (|W|, |Y1|, l) # <<<< this is the only non-cosmetic (variable-renaming) difference from the main version of this function
    if shape_info:
        print(f'|Q_l| = {my_Q_l.shape} = (|W|=n, |Y1|, l)')

#         print(f'{Y_prime.dtype}, {my_Q_l.dtype}')

    # NORMALIZATION
    V_prime = torch.einsum('mli,kil->mkl', Y_prime, my_Q_l)  # :: (m,n,l)
#         print(f"|V'| = {V_prime.shape} = (m, n, l)")
    M_prime = torch.prod(V_prime, 2) # :: (m,n)
#         print(f"|M'| = {M_prime.shape} = (m, n)")
#         N_prime = torch.matmul(M_prime, pX0f_torch) # :: (m, 1) <- prior probabilities of each of the m sampled channel prefixes
    N_prime = torch.matmul(M_prime, pW)
#         print(f"|N'| = {N_prime.shape} = (m, )")
    Z_prime = 1.0 / N_prime # :: (m, 1)
#         print(f"|Z'| = {Z_prime.shape} = (m, )")
    if shape_info:
        print(f"|V'| = {V_prime.shape} = (m, n, l)")
        print(f"|M'| = {M_prime.shape} = (m, n)")
        print(f"|N'| = {N_prime.shape} = (m, )")
        print(f"|Z'| = {Z_prime.shape} = (m, )")

    # NUMERATOR
    L_w = my_Q_l[xhat0f_idx]#.float()
    V_prime_w = torch.einsum('mij,ji->mi',Y_prime, L_w)
#         print(f"|V'_w| = {V_prime_w.shape} = (m, l)")
    O_w = torch.prod(V_prime_w, 1) # :: (m,1) likelihoods of each of the m sampled channel prefixes
#         print(f"|O_w| = {O_w.shape} = (m, )")
#         U_w = pX0f_torch[xhat0f_idx] * O_w ## :: (m,1) joint probabilities of xhat0f with each of the m sampled channel prefixes
    U_w = pW[xhat0f_idx] * O_w ## :: (m,1) joint probabilities of xhat0f with each of the m sampled channel prefixes
#         print(f"|U_w| = {U_w.shape} = (m, )")
    if shape_info:
        print(f"|V'_w| = {V_prime_w.shape} = (m, l)")
        print(f"|O_w| = {O_w.shape} = (m, )")
        print(f"|U_w| = {U_w.shape} = (m, )")


    if shape_info:
        E = torch.dot(Z_prime, U_w) / m
        print(f"|E| = scalar: type(E) = {type(E)}")
    return torch.dot(Z_prime, U_w) / m

def pXhat0fX0f_pxtn(xhat0f_idx, x0f_CM, c_idx, m = 50):
    return pXhat0fX0f_pxt(xhat0f_idx, x0f_CM, c_idx, m = 50).numpy()

def wordformTox0fCM(w=None, w_idx=None):
#     if w is None and w_idx is None:
#         raise Exception('At least one argument must be specified.')
    if w is None:
        w = Ws_t[w_idx]
        w_t = ds2t(w)
    else:
        w_t = ds2t(w)
        w_idx = Ws_t.index(w)
    
    total_x0f_length = len(w_t) #includes both edge symbols
#     word_length_noLE = total_x0f_length - 1 #still includes right edge symbol = "upcoming segment"
#     word_length_noRE = word_length_noLE - 1
#     offset = 2
#     assert word_length_produced_so_far - offset >= 0 , f"{word_length_produced_so_far} - {offset} < 0; w = {w}"
#     assert word_length_produced_so_far - offset < len(CMsByLengthByWordformIndex_torch), f"{word_length_produced_so_far} - {offset} >= {len(CMsByLengthByWordformIndex_torch)}; w = {w}"
#     assert w_idx in range(CMsByLengthByWordformIndex_torch[word_length_produced_so_far - offset].shape[0]), f"{w_idx}; w = {w}"
#     x0f_cm = CMsByLengthByWordformIndex_torch[word_length_produced_so_far - offset][w_idx]
    x0f_cm = exactCMsByLengthByWordformIndex_torch[total_x0f_length][w_idx]
    return x0f_cm

# for convenience
def pXhat0fX0f_pxtn_conv(w_hat, w, c, m = 50):
    w_idx = Ws_t.index(w_hat)

#     r_idx = Ps_t.index(r)
    source_wf = list(wordsWithPrefix(w, Ws))[0]
    source_wf_idx = Ws_t.index(source_wf)
    
    total_x0k_length = len(ds2t(w))# includes left edge symbol + upcoming segment
#     prefix_length_noWE = total_x0k_length - 1 #includes upcoming segment
#     prefix_length_produced_so_far = prefix_length_noWE - 1
#     offset = 1
#     x0k_cm = CMsByLengthByWordformIndex_torch[prefix_length_produced_so_far - offset][source_wf_idx]
    x0f_cm = exactCMsByLengthByWordformIndex_torch[total_x0k_length][source_wf_idx]
    
    c_idx = Cs_t.index(c)
    
    return pXhat0fX0f_pxtn(w_idx, x0f_cm, c_idx, m=m)

In [274]:
?phatXhat0fX0k_baseline

In [208]:
phatXhat0fX0k_baseline('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉',
                       '⋊.k.ɑ.n.t.æ.k.t.⋉.⋉',
                       'a couple of')

0.610786197167836

In [209]:
pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'),
               CMsByLengthByWordformIndex_torch[5][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')],
               Cs_t.index('a couple of'),
               m = 200)

tensor(0.0030)

In [280]:
pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'),
               CMsByLengthByWordformIndex_torch[9][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')],
               Cs_t.index('a couple of'),
               m = 50000)

tensor(0.6613)

In [276]:
pXhat0fX0i_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'),
               CMsByLengthByWordformIndex_torch[10][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')],
               Cs_t.index('a couple of'),
               m = 50000)

tensor(0.7017)

In [215]:
wordlengthsInclEdges
len(wordlengthsInclEdges)
len([None for each in range(min(wordlengthsInclEdges))])
len(exactCMsByLengthByWordformIndex_torch)

{4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19}

15

4

20

In [278]:
exactCMsByLengthByWordformIndex_torch[10].shape
exactCMsByLengthByWordformIndex_torch[10][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')].shape
exactCMsByLengthByWordformIndex_torch[10][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')]

torch.Size([9172, 39, 8])

torch.Size([39, 8])

tensor([[0.0439, 0.0181, 0.0091, 0.0090, 0.2037, 0.0093, 0.0111, 0.0000],
        [0.0070, 0.0137, 0.0091, 0.0090, 0.0075, 0.0093, 0.0111, 0.0000],
        [0.0255, 0.0072, 0.0091, 0.0090, 0.0188, 0.0093, 0.0111, 0.0000],
        [0.0070, 0.0071, 0.0090, 0.0171, 0.0074, 0.0092, 0.0108, 0.0000],
        [0.0068, 0.0069, 0.0170, 0.0087, 0.0072, 0.0090, 0.0105, 0.0000],
        [0.0070, 0.0072, 0.0091, 0.0090, 0.0710, 0.0351, 0.0111, 0.0000],
        [0.0068, 0.0070, 0.0089, 0.0088, 0.0073, 0.0091, 0.0108, 0.0000],
        [0.0070, 0.0072, 0.0091, 0.0090, 0.0075, 0.0178, 0.0111, 0.0000],
        [0.0068, 0.0071, 0.0090, 0.0089, 0.0074, 0.0176, 0.0111, 0.0000],
        [0.0070, 0.0072, 0.0091, 0.0090, 0.0075, 0.0093, 0.0111, 0.0000],
        [0.0052, 0.0062, 0.0078, 0.0077, 0.0064, 0.0080, 0.0111, 0.0000],
        [0.6525, 0.0071, 0.0090, 0.0089, 0.0074, 0.5897, 0.0403, 0.0000],
        [0.0255, 0.0072, 0.0091, 0.0090, 0.0075, 0.0093, 0.0111, 0.0000],
        [0.0070, 0.0071, 0.0090, 0.008

In [277]:
ds_l('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉') - 2
pXhat0fX0f_pxt(Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉'),
               exactCMsByLengthByWordformIndex_torch[10][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')],
               Cs_t.index('a couple of'),
               m = 50000)

8.0

tensor(0.7016)

In [224]:
exactCMsByLengthByWordformIndex_torch[10][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')] == CMsByLengthByWordformIndex_torch[10][Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')]

tensor([[1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1,

In [287]:
n_test_cases = 1000
random_wordform_idxs = choices(list(range(len(Ws_t))), k=n_test_cases)
random_context_idxs = choices(list(range(len(Cs_t))), k=n_test_cases)

In [292]:
%%timeit -n 100 -r 10

rand_w_idx = choice(random_wordform_idxs)
rand_c_idx = choice(random_context_idxs)

# ≈130-150ms on new sidious
pXhat0fX0f_pxt(rand_w_idx, wordformTox0fCM(w_idx=rand_w_idx), rand_c_idx, m = 50)

110 ms ± 4.85 ms per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [299]:
est_time_per_w_c = 110 #ms
len(Ws_t)
len(Cs_t)

serial_computation_time_est = est_time_per_w_c * len(Ws_t) * len(Cs_t) #ms
est_s = serial_computation_time_est / 1e3
est_m = est_s / 60
est_h = est_m / 60
est_d = est_h / 24

est_at_scales = (est_s, est_m, est_h, est_d)

toSN = lambda n: "{0:.2E}".format(n)

tuple(map(toSN,
          est_at_scales))

9172

106295

('1.07E+08', '1.79E+06', '2.98E+04', '1.24E+03')

## Calculation volume organization for Case 1

See Section 6.2 for explanation of the "cases".

### Slices: Calculate results for groups of reconstructed wordforms by (input prefix, context)

In [225]:
len(Ps_t)

42231

In [226]:
def retrieveKcousins(r, maxK):
    word_cousin_indices = union([set(list(cousin_mats[k][Ps_t.index(r),:].nonzero()[0])) for k in range(maxK)])
    word_cousins = list(map(lambda w_idx: Ws_t[w_idx], word_cousin_indices))
    return word_cousins

In [227]:
retrieveKcousins('⋊.k.ɑ.n.t', 2)

['⋊.k.ɑ.ɹ.t.n.⋉.⋉',
 '⋊.k.ɑ.ɹ.t.u.n.⋉.⋉',
 '⋊.k.ɑ.ɹ.t.ɛ.k.s.⋉.⋉',
 '⋊.k.ɑ.ɹ.t.ɛ.l.⋉.⋉',
 '⋊.k.ɑ.ɹ.t.ɹ.ɪ.dʒ.⋉.⋉',
 '⋊.k.ɑ.ɹ.t.⋉.⋉',
 '⋊.k.aʊ.n.t.i.⋉.⋉',
 '⋊.k.aʊ.n.t.l.ə.s.⋉.⋉',
 '⋊.k.aʊ.n.t.ɚ.b.æ.l.ɪ.n.s.⋉.⋉',
 '⋊.k.aʊ.n.t.ɚ.f.ɪ.t.⋉.⋉',
 '⋊.k.aʊ.n.t.ɚ.æ.k.t.⋉.⋉',
 '⋊.k.aʊ.n.t.ɚ.⋉.⋉',
 '⋊.k.aʊ.n.t.⋉.⋉',
 '⋊.h.ɑ.n.t.⋉.⋉',
 '⋊.k.æ.n.t.n.⋉.⋉',
 '⋊.k.æ.n.t.ə.l.oʊ.p.⋉.⋉',
 '⋊.k.æ.n.t.ɚ.⋉.⋉',
 '⋊.k.æ.n.t.⋉.⋉',
 '⋊.dʒ.ɑ.n.t.⋉.⋉',
 '⋊.k.ʌ.n.t.ɹ.i.⋉.⋉',
 '⋊.k.ə.n.t.eɪ.n.⋉.⋉',
 '⋊.k.ə.n.t.æ.m.ə.n.eɪ.t.⋉.⋉',
 '⋊.k.ə.n.t.æ.m.ə.n.ɪ.n.t.⋉.⋉',
 '⋊.k.ə.n.t.ɛ.m.p.t.⋉.⋉',
 '⋊.k.ə.n.t.ɛ.n.d.⋉.⋉',
 '⋊.k.ə.n.t.ɛ.n.t.ɪ.d.⋉.⋉',
 '⋊.k.ə.n.t.ɛ.n.t.⋉.⋉',
 '⋊.w.ɑ.n.t.⋉.⋉',
 '⋊.k.ə.n.t.ɛ.n.tʃ.ɪ.n.⋉.⋉',
 '⋊.k.ə.n.t.ɪ.g.j.u.ə.s.⋉.⋉',
 '⋊.k.ə.n.t.ɪ.n.dʒ.ɪ.n.s.i.⋉.⋉',
 '⋊.k.ə.n.t.ɪ.n.j.u.eɪ.ʃ.ɪ.n.⋉.⋉',
 '⋊.k.ə.n.t.ɪ.n.j.u.ə.s.⋉.⋉',
 '⋊.k.ə.n.t.ɪ.n.j.u.⋉.⋉',
 '⋊.k.ə.n.t.ɹ.oʊ.l.⋉.⋉',
 '⋊.k.ə.n.t.ɹ.æ.k.ʃ.ɪ.n.⋉.⋉',
 '⋊.k.ə.n.t.ɹ.æ.p.ʃ.ɪ.n.⋉.⋉',
 '⋊.k.ə.n.t.ɹ.ɪ.b.j.u.t.ɚ.⋉.⋉',
 '⋊.k.ə.n.t.ɹ.ɪ.b.j.u.t.⋉.⋉',
 '⋊.k.ɑ.n.d.u.ɪ.

In [228]:
cousin_mats[0].shape
cousin_mats[0][Ps_t.index('⋊.k.ɑ.n.t'), Ws_t.index('⋊.k.ɑ.n.t.æ.k.t.⋉.⋉')] #should be 1

(42231, 9172)

1

In [229]:
wordsWithPrefix('⋊.k.ɑ.n.t', Ws)

{'⋊.k.ɑ.n.t.n.u.ɪ.t.i.⋉.⋉',
 '⋊.k.ɑ.n.t.n.ɪ.n.t.⋉.⋉',
 '⋊.k.ɑ.n.t.æ.k.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ə.m.p.l.eɪ.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɛ.k.s.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɛ.s.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.k.t.ɚ.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.k.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.æ.s.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.b.j.u.ʃ.ɪ.n.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.b.æ.n.d.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.d.ɪ.k.t.⋉.⋉',
 '⋊.k.ɑ.n.t.ɹ.ə.v.ɚ.s.i.⋉.⋉'}

In [230]:
def get_nnz_word(my_k, j):
    return cousin_mats[my_k][:,j].nnz
    
cousin_mats[1].shape
word_nnz_counts_by_k = {my_k:par(delayed(get_nnz_word)(my_k, j) for j in range(cousin_mats[my_k].shape[1]))
                        for my_k in range(5)}

(42231, 9172)

[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Batch computation too fast (0.0139s.) Setting batch_size=28.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Batch computation too fast (0.1015s.) Setting batch_size=110.
[Parallel(n_jobs=16)]: Done 256 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done 620 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 1148 tasks      | elapsed:    0.7s
[Parallel(n_jobs=16)]: Done 2798 tasks      | elapsed:    1.1s
[Parallel(n_jobs=16)]: Done 4448 tasks      | elapsed:    1.7s
[Parallel(n_jobs=16)]: Done 9172 out of 9172 | elapsed:    2.8s finished
[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Batch computation too fast (0.0283s.) Setting batch_size=14.

[Parallel(n_jobs=16)]: Done 5268 tasks      | elapsed:   16.4s
[Parallel(n_jobs=16)]: Done 5488 tasks      | elapsed:   17.0s
[Parallel(n_jobs=16)]: Done 5708 tasks      | elapsed:   17.6s
[Parallel(n_jobs=16)]: Done 5936 tasks      | elapsed:   18.3s
[Parallel(n_jobs=16)]: Done 6164 tasks      | elapsed:   19.3s
[Parallel(n_jobs=16)]: Done 6400 tasks      | elapsed:   19.9s
[Parallel(n_jobs=16)]: Done 6636 tasks      | elapsed:   20.5s
[Parallel(n_jobs=16)]: Done 6880 tasks      | elapsed:   21.2s
[Parallel(n_jobs=16)]: Done 7124 tasks      | elapsed:   21.9s
[Parallel(n_jobs=16)]: Done 7376 tasks      | elapsed:   22.6s
[Parallel(n_jobs=16)]: Done 7628 tasks      | elapsed:   23.4s
[Parallel(n_jobs=16)]: Done 7888 tasks      | elapsed:   24.3s
[Parallel(n_jobs=16)]: Done 8148 tasks      | elapsed:   24.9s
[Parallel(n_jobs=16)]: Done 8416 tasks      | elapsed:   25.7s
[Parallel(n_jobs=16)]: Done 8684 tasks      | elapsed:   26.2s
[Parallel(n_jobs=16)]: Done 8960 tasks      | elapsed: 

In [231]:
from statistics import mean, median

In [232]:
for my_k in range(5):
    print(f"max number of {my_k}-cousins = {max(word_nnz_counts_by_k[my_k])}")
    print(f"median number of {my_k}-cousins = {median(word_nnz_counts_by_k[my_k])}")
    print(f"mean number of {my_k}-cousins = {mean(word_nnz_counts_by_k[my_k])}")
    print(' ')

max number of 0-cousins = 19
median number of 0-cousins = 8.0
mean number of 0-cousins = 8.692433493240296
 
max number of 1-cousins = 181
median number of 1-cousins = 109.0
mean number of 1-cousins = 109.03576101177497
 
max number of 2-cousins = 1767
median number of 2-cousins = 1001.0
mean number of 2-cousins = 1014.8246838203228
 
max number of 3-cousins = 6517
median number of 3-cousins = 4110.0
mean number of 3-cousins = 4341.6341037941565
 
max number of 4-cousins = 11295
median number of 4-cousins = 7275.0
mean number of 4-cousins = 7789.9738334060185
 


In [233]:
def get_nnz_prefix(my_k, i):
    return cousin_mats[my_k][i,:].nnz
    
cousin_mats[1].shape
prefix_nnz_counts_by_k = {my_k:par(delayed(get_nnz_prefix)(my_k, i) for i in range(cousin_mats[my_k].shape[0]))
                          for my_k in range(5)}

(42231, 9172)

[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Batch computation too fast (0.0077s.) Setting batch_size=52.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Batch computation too fast (0.0155s.) Setting batch_size=1340.
[Parallel(n_jobs=16)]: Done 448 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 1124 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done 42231 out of 42231 | elapsed:    0.7s finished
[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Batch computation too fast (0.0083s.) Setting batch_size=48.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:    0.0

In [234]:
for my_k in range(5):
    print(f"max number of {my_k}-cousins = {max(prefix_nnz_counts_by_k[my_k])}")
    print(f"median number of {my_k}-cousins = {median(prefix_nnz_counts_by_k[my_k])}")
    print(f"mean number of {my_k}-cousins = {mean(prefix_nnz_counts_by_k[my_k])}")
    print(' ')

max number of 0-cousins = 9172
median number of 0-cousins = 1
mean number of 0-cousins = 1.8878785726125358
 
max number of 1-cousins = 9171
median number of 1-cousins = 1
mean number of 1-cousins = 23.681087352892426
 
max number of 2-cousins = 9162
median number of 2-cousins = 10
mean number of 2-cousins = 220.40614714309393
 
max number of 3-cousins = 9014
median number of 3-cousins = 79
mean number of 3-cousins = 942.9439984845255
 
max number of 4-cousins = 8940
median number of 4-cousins = 278
mean number of 4-cousins = 1691.8765835523668
 


In [235]:
cousin_mats[1].shape

(42231, 9172)

In [236]:
def to_prefix_idx(w):
    return Ps_t.index(w)

prefix_indices_of_full_wordforms = par(delayed(to_prefix_idx)(w) for w in Ws)

[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Batch computation too fast (0.0067s.) Setting batch_size=60.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Batch computation too fast (0.0434s.) Setting batch_size=552.
[Parallel(n_jobs=16)]: Done 512 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 9172 out of 9172 | elapsed:    0.5s finished


In [237]:
def to_word_idx(p):
    return Ws_t.index(p)

def get_wordform_idx(prefix_idx):
    return to_word_idx(Ps_t[prefix_idx])

# get_wordform_idx = {prefix_idx:to_word_idx(Ps_t[prefix_idx])
#                     for prefix_idx in prefix_indices_of_full_wordforms}

In [238]:
Ps_t[8370]
get_wordform_idx(8370)
Ws_t[get_wordform_idx(8370)]

'⋊.h.aɪ.k.⋉.⋉'

1928

'⋊.h.aɪ.k.⋉.⋉'

In [239]:
cousin_mats[k].shape

(42231, 9172)

In [240]:
def getKcousin_idxs_of_wordform(w_idx, maxK, asType=None, onlyWordCousins = False, getPrefixIndices=True):
    kcousin_indices = union([set(list(cousin_mats[k][:,w_idx].nonzero()[0])) for k in range(maxK)])
    if onlyWordCousins:
    # top line is ≈100x slower
#         kcousin_indices = {prefix_idx for prefix_idx in kcousin_indices if prefix_idx in prefix_indices_of_full_wordforms}
        kcousin_indices = set.intersection(kcousin_indices, prefix_indices_of_full_wordforms)
    if not getPrefixIndices:
        kcousin_indices = set(map(get_wordform_idx, kcousin_indices))
    
    if asType == 'ndarray':
        kcousin_indices = np.array(sorted(list(kcousin_indices)), dtype='int32')
        return kcousin_indices
    elif asType == 'torch':
        kcousin_indices = torch.tensor(sorted(list(kcousin_indices)), dtype=torch.int32)
        return kcousin_indices
    else:
        return kcousin_indices
    
def getKcousin_idxs_of_prefix(p_idx, maxK, asType=None):
    kcousin_indices = union([set(list(cousin_mats[k][p_idx].nonzero()[0])) for k in range(maxK)])
    
    if asType == 'ndarray':
        kcousin_indices = np.array(sorted(list(kcousin_indices)), dtype='int32')
        return kcousin_indices
    elif asType == 'torch':
        kcousin_indices = torch.tensor(sorted(list(kcousin_indices)), dtype=torch.int32)
        return kcousin_indices
    else:
        return kcousin_indices

In [241]:
random_source_wordform
getKcousin_idxs_of_wordform(Ws_t.index(random_source_wordform), 2, 'ndarray', True, False)

'⋊.tʃ.ɛ.k.ɚ.⋉.⋉'

array([6556, 6560, 6561], dtype=int32)

In [242]:
random_source_prefix
getKcousin_idxs_of_prefix(Ps_t.index(random_source_prefix), 2, 'ndarray')

'⋊.ɪ.d.ɪ.ʃ.ɪ'

array([7256, 7476, 7885, 8349], dtype=int32)

In [243]:
def pXhat0fX0i_pxtn_by_rc(produced_prefix_idx, x0k_cm, c_idx, m = 50, asType='ndarray', parallel=False):
    kCousin_idxs = getKcousin_idxs_of_prefix(produced_prefix_idx, k, 'ndarray')
    
    if x0k_cm is None:
        x0k_cm = prefixTox0kCM(Ps_t[produced_prefix_idx])
    
    if parallel:
        estimates = par(delayed(pXhat0fX0i_pxtn)(w_idx, x0k_cm, c_idx, m=m) for w_idx in kCousin_idxs)
        estimates = np.array(list(estimates))
    else:
        estimates = np.array([pXhat0fX0i_pxtn(w_idx, x0k_cm, c_idx, m=m) for w_idx in kCousin_idxs])

    if asType == 'ndarray':
        return kCousin_idxs, estimates
    elif asType == 'torch':
#         return torch.tensor(kCousin_idxs), torch.tensor(estimates)
        return torch.from_numpy(kCousin_idxs), torch.from_numpy(estimates)
    else:
        return tuple(kCousin_idxs), tuple(estimates)

def pXhat0fX0i_pxtn_by_wc(produced_wordform_idx, x0k_cm, c_idx, m = 50, asType='ndarray', parallel=False):
    equivalent_prefix_idx = Ps_t.index(Ws_t[produced_wordform_idx])
    return pXhat0fX0i_pxtn_by_rc(equivalent_prefix_idx, x0k_cm, c_idx, m, asType, parallel)

In [244]:
random_source_prefix
random_source_prefix_idx = Ps_t.index(random_source_prefix)
random_source_prefix_x0kcm = prefixTox0kCM(random_source_prefix)

'⋊.ɪ.d.ɪ.ʃ.ɪ'

In [245]:
print(f'x0k = {random_source_prefix}')
print(f'c = {random_context}')
cousin_idxs, post_probs = pXhat0fX0i_pxtn_by_rc(random_source_prefix_idx, random_source_prefix_x0kcm, random_context_idx)
print(f'({k}-cousin W-hat, p(W-hat|X0k = {random_source_prefix}, C = {random_context}))')
[(Ws_t[cousin_idx], post_probs[i]) for i, cousin_idx in enumerate(cousin_idxs)]

x0k = ⋊.ɪ.d.ɪ.ʃ.ɪ
c = a couple of
(2-cousin W-hat, p(W-hat|X0k = ⋊.ɪ.d.ɪ.ʃ.ɪ, C = a couple of))


[('⋊.ɑ.d.ɪ.ʃ.ɪ.n.⋉.⋉', 0.0021512415378222975),
 ('⋊.ə.d.ɪ.ʃ.ɪ.n.⋉.⋉', 0.0092779551698348398),
 ('⋊.ɪ.d.ɪ.ʃ.ɪ.n.⋉.⋉', 0.018404715597030011),
 ('⋊.ɪ.n.ɪ.ʃ.ɪ.t.ɪ.v.⋉.⋉', 0.00066706208380561984)]

### Blocks: Calculate results for groups of input prefixes by context

In [246]:
#FIXME

## Calculation volume organization 2

See Section 6.2 for explanation of the "cases".

In [247]:
#FIXME

## Calculation volume organization 3

See Section 6.2 for explanation of the "cases".

### Slices: Calculate $\{p(\widehat{W} = w^* | W = w^*, C = c)\}$ for all $W$ and for a single given $c$ 

In [248]:
{w for w in Ws_t if ds2t(w)[0] != leftEdge or ds2t(w)[-1] != rightEdge}

set()

In [249]:
Ws_t.index('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉.⋉')
len(ds2t('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉.⋉')) #19

6304

19

In [271]:
len(CMsByLengthByWordformIndex_torch)
CMsByLengthByWordformIndex_torch[19][6304].shape
CMsByLengthByWordformIndex_torch[19][6304]

20

torch.Size([39, 17])

tensor([[0.0070, 0.0366, 0.0071, 0.0071, 0.0092, 0.0176, 0.0114, 0.0093, 0.0070,
         0.0091, 0.0159, 0.0089, 0.0072, 0.0071, 0.0075, 0.0070, 0.0000],
        [0.0070, 0.0089, 0.0071, 0.0071, 0.0092, 0.0335, 0.0114, 0.0093, 0.0070,
         0.0091, 0.0159, 0.0089, 0.0072, 0.0071, 0.0075, 0.0070, 0.0000],
        [0.0070, 0.0089, 0.0135, 0.0071, 0.0092, 0.0176, 0.0114, 0.0093, 0.0070,
         0.0091, 0.0159, 0.0089, 0.0072, 0.0071, 0.0075, 0.0070, 0.0000],
        [0.0070, 0.0088, 0.0070, 0.0070, 0.0091, 0.0174, 0.0113, 0.0092, 0.0069,
         0.0090, 0.0157, 0.0088, 0.0071, 0.0070, 0.0142, 0.0068, 0.0000],
        [0.0068, 0.0086, 0.0068, 0.0068, 0.0088, 0.0169, 0.0110, 0.0089, 0.0068,
         0.0088, 0.0153, 0.0086, 0.0069, 0.0068, 0.0072, 0.0066, 0.0000],
        [0.0070, 0.0550, 0.0071, 0.2747, 0.0092, 0.1082, 0.0114, 0.0093, 0.0070,
         0.0091, 0.1078, 0.0089, 0.7089, 0.0071, 0.1108, 0.0070, 0.0000],
        [0.0068, 0.0087, 0.0069, 0.0069, 0.0090, 0.0171, 0.0111, 0.009

In [251]:
len(ds2t('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ'))
len(dsToKfactorSequence(3, '⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ'))
prefixTox0kCM('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ').shape

len(ds2t('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n'))
len(dsToKfactorSequence(3, '⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n'))
prefixTox0kCM('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n').shape

len(ds2t('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉'))
len(dsToKfactorSequence(3, '⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉'))
prefixTox0kCM('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉').shape

len(ds2t('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉.⋉'))
len(dsToKfactorSequence(3, '⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉.⋉'))
prefixTox0kCM('⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n.⋉.⋉').shape

16

14

torch.Size([39, 14])

17

15

...

16

torch.Size([39, 16])

19

17

torch.Size([39, 17])

In [252]:
len(ds2t("⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n"))
prefixTox0kCM("⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n").shape
prefixTox0kCM("⋊.t.ɛ.l.ɪ.k.ə.m.j.u.n.ə.k.eɪ.ʃ.ɪ.n")

17

torch.Size([39, 15])

tensor([[0.0070, 0.0366, 0.0071, 0.0071, 0.0092, 0.0176, 0.0114, 0.0093, 0.0070,
         0.0091, 0.0159, 0.0089, 0.0072, 0.0071, 0.0075],
        [0.0070, 0.0089, 0.0071, 0.0071, 0.0092, 0.0335, 0.0114, 0.0093, 0.0070,
         0.0091, 0.0159, 0.0089, 0.0072, 0.0071, 0.0075],
        [0.0070, 0.0089, 0.0135, 0.0071, 0.0092, 0.0176, 0.0114, 0.0093, 0.0070,
         0.0091, 0.0159, 0.0089, 0.0072, 0.0071, 0.0075],
        [0.0070, 0.0088, 0.0070, 0.0070, 0.0091, 0.0174, 0.0113, 0.0092, 0.0069,
         0.0090, 0.0157, 0.0088, 0.0071, 0.0070, 0.0142],
        [0.0068, 0.0086, 0.0068, 0.0068, 0.0088, 0.0169, 0.0110, 0.0089, 0.0068,
         0.0088, 0.0153, 0.0086, 0.0069, 0.0068, 0.0072],
        [0.0070, 0.0550, 0.0071, 0.2747, 0.0092, 0.1082, 0.0114, 0.0093, 0.0070,
         0.0091, 0.1078, 0.0089, 0.7089, 0.0071, 0.1108],
        [0.0068, 0.0087, 0.0069, 0.0069, 0.0090, 0.0171, 0.0111, 0.0090, 0.0068,
         0.0089, 0.0155, 0.0087, 0.0070, 0.0069, 0.0073],
        [0.0070, 0.0089, 0.

In [ ]:
#BOOKMARK RESUME

In [253]:
# CMsByLengthByWordformIndex_torch
# X0f_CMs_by_wordform_index = [wordformTox0kCM(w, w_idx) for w_idx, w in enumerate(Ws_t)]
# X0f_CMs_by_wordform_index_torch = torch.tensor([wordformTox0kCM(w, w_idx) for w_idx, w in enumerate(Ws_t)])
# X0f_CMs_by_wordform_index_torch.shape

In [316]:
def p_wstar_wstar_pxtn_by_wc(produced_wordform_idx, x0f_cm, c_idx, m = 50):
    return pXhat0fX0f_pxtn(produced_wordform_idx, x0f_cm, c_idx, m = m)
#     return pXhat0fX0i_pxtn(produced_wordform_idx, x0f_cm, c_idx, m = m)

def p_wstar_wstar_pxtn_by_c(c_idx, m, asType='ndarray', parallel=True):
    if parallel:
        estimates = np.array(list(par(delayed(p_wstar_wstar_pxtn_by_wc)(w_idx, 
                                                                        wordformTox0fCM(w_idx=w_idx),
                                                                        c_idx,
                                                                        m)
                                      for w_idx in range(len(Ws_t)))))
    else:
        estimates = np.array([p_wstar_wstar_pxtn_by_wc(w_idx, 
                                                       wordformTox0fCM(w_idx=w_idx),
                                                       c_idx,
                                                       m)
                             for w_idx in np.arange(len(Ws_t))])
    
    if asType =='ndarray':
        return estimates
    elif asType == 'torch':
        return torch.tensor(estimates)
    else:
        return tuple(estimates)
    
def p_wstar_wstar_pxt_by_wc(produced_wordform_idx, x0f_cm, c_idx, m = 50):
    return pXhat0fX0f_pxt(produced_wordform_idx, x0f_cm, c_idx, m = m)
#     return pXhat0fX0i_pxtn(produced_wordform_idx, x0f_cm, c_idx, m = m)

def p_wstar_wstar_pxt_by_c(c_idx, m, asType='torch', parallel=True):
    if parallel:
        estimates = torch.tensor(list(par(delayed(p_wstar_wstar_pxt_by_wc)(w_idx, 
                                                                           wordformTox0fCM(w_idx=w_idx),
                                                                           c_idx,
                                                                           m)
                                          for w_idx in range(len(Ws_t)))))
    else:
        estimates = torch.tensor([p_wstar_wstar_pxt_by_wc(w_idx,
                                                          wordformTox0fCM(w_idx=w_idx),
                                                          c_idx,
                                                          m)
                                  for w_idx in np.arange(len(Ws_t))])
    
    if asType =='ndarray':
        return estimates.numpy()
    elif asType == 'torch':
        return estimates
    else:
        return tuple(estimates)

In [256]:
random_source_wordform
random_source_wordform_idx = Ws_t.index(random_source_wordform)
random_source_wordform_x0f_cm = wordformTox0fCM(random_source_wordform, random_source_wordform_idx)
random_source_wordform_x0f_cm.shape
random_source_wordform_x0f_cm

'⋊.tʃ.ɛ.k.ɚ.⋉.⋉'

torch.Size([39, 5])

tensor([[0.0070, 0.0075, 0.0071, 0.0070, 0.0000],
        [0.0070, 0.0075, 0.0071, 0.0070, 0.0000],
        [0.0070, 0.0075, 0.0071, 0.0070, 0.0000],
        [0.0070, 0.0074, 0.0070, 0.0068, 0.0000],
        [0.0068, 0.0072, 0.0068, 0.0066, 0.0000],
        [0.0070, 0.0915, 0.0071, 0.0070, 0.0000],
        [0.0068, 0.0073, 0.0069, 0.0068, 0.0000],
        [0.0070, 0.0075, 0.0178, 0.0070, 0.0000],
        [0.0068, 0.0074, 0.0135, 0.0070, 0.0000],
        [0.0070, 0.0143, 0.0071, 0.0070, 0.0000],
        [0.0052, 0.0123, 0.0061, 0.0070, 0.0000],
        [0.0068, 0.0141, 0.7177, 0.0070, 0.0000],
        [0.0070, 0.0075, 0.0071, 0.0070, 0.0000],
        [0.0070, 0.0074, 0.0070, 0.0068, 0.0000],
        [0.0070, 0.0075, 0.0071, 0.0070, 0.0000],
        [0.0070, 0.0075, 0.0071, 0.0070, 0.0000],
        [0.0070, 0.0074, 0.0070, 0.0068, 0.0000],
        [0.0068, 0.0074, 0.0070, 0.0070, 0.0000],
        [0.0255, 0.0075, 0.0071, 0.0070, 0.0000],
        [0.7264, 0.0075, 0.0071, 0.0070, 0.0000],


In [306]:
J

-1

In [307]:
! free -h

              total        used        free      shared  buff/cache   available
Mem:           188G         11G        119G        2.9M         57G        175G
Swap:          2.0G          0B        2.0G


In [302]:
random_c_idx1 = choice(random_context_idxs); Cs_t[random_c_idx1]

'made you'

In [308]:
random_slice1 = p_wstar_wstar_pxtn_by_c(random_c_idx1, m=50, asType='ndarray', parallel=True)
random_slice1.shape
random_slice1.nbytes / 1e6
random_slice1.nbytes / 1e9

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    3.9s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 117 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 178 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 201 tasks      | elapsed:    9.2s
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 249 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done 274 tasks      |

array([  2.69631086e-02,   4.16203889e-01,   7.87390956e-01, ...,
         8.54681841e-05,   3.27203296e-05,   4.60355938e-04])

In [309]:
random_c_idx2 = choice(random_context_idxs); Cs_t[random_c_idx2]

'well yeah what'

In [310]:
random_slice2 = p_wstar_wstar_pxtn_by_c(random_c_idx2, m=50, asType='ndarray', parallel=True)
random_slice2.shape
random_slice2.nbytes / 1e6
random_slice2.nbytes / 1e9

[Parallel(n_jobs=-1)]: Using backend MultiprocessingBackend with 32 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    5.2s
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done 117 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done 178 tasks      | elapsed:    8.7s
[Parallel(n_jobs=-1)]: Done 201 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 224 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 249 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done 274 tasks      |

(9172,)

0.073376

7.3376e-05

In [312]:
J = 4

In [313]:
random_c_idx3 = choice(random_context_idxs); Cs_t[random_c_idx3]

'the source'

In [314]:
random_slice3 = p_wstar_wstar_pxtn_by_c(random_c_idx3, m=50, asType='ndarray', parallel=True)
random_slice3.shape
random_slice3.nbytes / 1e6
random_slice3.nbytes / 1e9

[Parallel(n_jobs=4)]: Using backend MultiprocessingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.9s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    1.4s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    1.8s
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    2.3s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    2.6s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:    3.2s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    3.8s
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:    4.5s
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:    5.1s
[Parallel(n_jobs=4)]: Done 105 tasks      | elapsed:    5.7s
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed:    6.2s
[Parallel(n_jobs=4)]: Done 137 tasks      | elapsed:    6.7s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    7.6s
[Parallel(n_jobs=4)]: Done 173 tasks      | elapsed:    8.2s

(9172,)

0.073376

7.3376e-05

In [315]:
J = 8

In [317]:
random_c_idx4 = choice(random_context_idxs); Cs_t[random_c_idx4]

'i i like'

In [318]:
random_slice4 = p_wstar_wstar_pxtn_by_c(random_c_idx4, m=50, asType='ndarray', parallel=True)
random_slice4.shape
random_slice4.nbytes / 1e6
random_slice4.nbytes / 1e9

[Parallel(n_jobs=8)]: Using backend MultiprocessingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done  16 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    1.9s
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    2.2s
[Parallel(n_jobs=8)]: Done  45 tasks      | elapsed:    2.6s
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    3.0s
[Parallel(n_jobs=8)]: Done  69 tasks      | elapsed:    3.4s
[Parallel(n_jobs=8)]: Done  82 tasks      | elapsed:    3.8s
[Parallel(n_jobs=8)]: Done  97 tasks      | elapsed:    4.7s
[Parallel(n_jobs=8)]: Done 112 tasks      | elapsed:    5.1s
[Parallel(n_jobs=8)]: Done 129 tasks      | elapsed:    5.5s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    6.3s
[Parallel(n_jobs=8)]: Done 165 tasks      | elapsed:    6.8s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    7.4s

(9172,)

0.073376

7.3376e-05

In [319]:
J = 16

In [320]:
random_c_idx5 = choice(random_context_idxs); Cs_t[random_c_idx5]

'but what about'

In [321]:
random_slice5 = p_wstar_wstar_pxtn_by_c(random_c_idx5, m=50, asType='ndarray', parallel=True)
random_slice5.shape
random_slice5.nbytes / 1e6
random_slice5.nbytes / 1e9

[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    1.2s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    1.8s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:    2.4s
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    2.8s
[Parallel(n_jobs=16)]: Done  53 tasks      | elapsed:    3.3s
[Parallel(n_jobs=16)]: Done  66 tasks      | elapsed:    4.0s
[Parallel(n_jobs=16)]: Done  81 tasks      | elapsed:    4.4s
[Parallel(n_jobs=16)]: Done  96 tasks      | elapsed:    5.2s
[Parallel(n_jobs=16)]: Done 113 tasks      | elapsed:    5.7s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:    6.1s
[Parallel(n_jobs=16)]: Done 149 tasks      | elapsed:    6.9s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    7.6s
[Parallel(n_jobs=16)]: Done 189 tasks      | elapsed:    8.4s
[Parallel(n_jobs=16)]: Done 210 tasks      | elapsed:    9.1s
[Parallel(n_jobs=16)]: Done 233 tasks      |

(9172,)

0.073376

7.3376e-05

In [323]:
random_c_idx6 = random_c_idx5

random_slice6 = p_wstar_wstar_pxt_by_c(random_c_idx6, m=50, asType='torch', parallel=True)
random_slice6.shape
torch_nbytes(random_slice6) / 1e6
torch_nbytes(random_slice6) / 1e9

[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    1.2s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    1.8s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:    2.4s
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    2.8s
[Parallel(n_jobs=16)]: Done  53 tasks      | elapsed:    3.4s
[Parallel(n_jobs=16)]: Done  66 tasks      | elapsed:    4.1s
[Parallel(n_jobs=16)]: Done  81 tasks      | elapsed:    4.6s
[Parallel(n_jobs=16)]: Done  96 tasks      | elapsed:    5.4s
[Parallel(n_jobs=16)]: Done 113 tasks      | elapsed:    5.9s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:    6.3s
[Parallel(n_jobs=16)]: Done 149 tasks      | elapsed:    7.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    7.6s
[Parallel(n_jobs=16)]: Done 189 tasks      | elapsed:    8.3s
[Parallel(n_jobs=16)]: Done 210 tasks      | elapsed:    9.0s
[Parallel(n_jobs=16)]: Done 233 tasks      |

torch.Size([9172])

AttributeError: 'Tensor' object has no attribute 'nbytes'

In [324]:
random_c_idx6 = random_c_idx5

random_slice6 = p_wstar_wstar_pxt_by_c(random_c_idx6, m=50, asType='torch', parallel=True)
random_slice6.shape
torch_nbytes(random_slice6) / 1e6
torch_nbytes(random_slice6) / 1e9

[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    1.3s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    1.9s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:    2.4s
[Parallel(n_jobs=16)]: Done  40 tasks      | elapsed:    2.8s
[Parallel(n_jobs=16)]: Done  53 tasks      | elapsed:    3.3s
[Parallel(n_jobs=16)]: Done  66 tasks      | elapsed:    3.9s
[Parallel(n_jobs=16)]: Done  81 tasks      | elapsed:    4.5s
[Parallel(n_jobs=16)]: Done  96 tasks      | elapsed:    5.2s
[Parallel(n_jobs=16)]: Done 113 tasks      | elapsed:    5.9s
[Parallel(n_jobs=16)]: Done 130 tasks      | elapsed:    6.3s
[Parallel(n_jobs=16)]: Done 149 tasks      | elapsed:    7.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    7.7s
[Parallel(n_jobs=16)]: Done 189 tasks      | elapsed:    8.4s
[Parallel(n_jobs=16)]: Done 210 tasks      | elapsed:    9.1s
[Parallel(n_jobs=16)]: Done 233 tasks      |

torch.Size([9172])

AttributeError: 'Tensor' object has no attribute 'nbytes'

# Calculate distribution

## Number of calculations necessary under different calculation criteria

In general, the full distribution that could be calculated is $$p(\widehat{X}_0^f | X_0^k, C) = p(\widehat{W} | R, C)$$

In [257]:
len(Ws)
len(Ps)
len(Cs)
"{0:,}".format( len(Ws) * len(Ps) * len(Cs) )
"{0:.2E}".format( len(Ws) * len(Ps) * len(Cs) )

9172

42231

106295

'41,172,595,697,940'

'4.12E+13'

For the chosen value of $k$, there are this many $(w,r)$ pairs to be calculated (for every $c$):

In [258]:
# cousin_mats[k].shape
# sum([cousin_mats[k][i,:].nnz for i in range(len(Ps))])

In [259]:
print(f'k = {k}')

cousin_mats[k].shape

def calc_count(i):
    return cousin_mats[k][i,:].nnz

num_seq_pairs_case1 = sum(list(par(delayed(calc_count)(i) for i in range(len(Ps)))))
num_calcs_case1 = len(Cs) * num_seq_pairs_case1

print('|(w,r)| pairs to do calculations for every c = {0:,}'.format( num_seq_pairs_case1 ))
print("|(w,r)| pairs there would be without the k-edit-distance-restriction = {0:,}".format( len(Ws) * len(Ps) ))
print(f'|Cs| = {len(Cs)}')
print('Est. of total number of calculations to do = {0:,} = {0:.2E}'.format(num_calcs_case1))

k = 2


(42231, 9172)

[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Batch computation too fast (0.0088s.) Setting batch_size=44.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Batch computation too fast (0.0202s.) Setting batch_size=872.
[Parallel(n_jobs=16)]: Done 384 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 956 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Batch computation too fast (0.1777s.) Setting batch_size=1962.
[Parallel(n_jobs=16)]: Done 3184 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 42231 out of 42231 | elapsed:    0.7s finished


|(w,r)| pairs to do calculations for every c = 9,307,972
|(w,r)| pairs there would be without the k-edit-distance-restriction = 387,342,732
|Cs| = 106295
Est. of total number of calculations to do = 989,390,883,740 = 9.89E+11


If $r$ is `False`, then the dimensions and number of calculations reduce to:

In [260]:
len(Ws)
len(Ws)
len(Cs)
"{0:,}".format( len(Ws) * len(Ws) * len(Cs) )
"{0:.2E}".format( len(Ws) * len(Ws) * len(Cs) )

9172

9172

106295

'8,942,128,951,280'

'8.94E+12'

If $r$ is `False`, then for the chosen value of $k$, there are this many $(w',w)$ pairs to be calculated (for every $c$):

In [261]:
print(f'k = {k}')

cousin_mats[k].shape

def calc_count(i):
    if Ps_t[i] in Ws:
        return cousin_mats[k][i,:].nnz
    else:
        return 0

num_seq_pairs_case2 = sum(list(par(delayed(calc_count)(i) for i in range(len(Ps)))))
num_calcs_case2 = len(Cs) * num_seq_pairs_case2

print("|(w',w)| pairs to do calculations for every c = {0:,}".format( num_seq_pairs_case2 ))
print("|(w',w)| pairs there would be without the k-edit-distance-restriction = {0:,}".format( len(Ws) * len(Ws) ))
print(f'|Cs| = {len(Cs)}')
print('Est. of total number of calculations to do = {0:,} = {0:.2E}'.format(num_calcs_case2))

k = 2


(42231, 9172)

[Parallel(n_jobs=16)]: Using backend MultiprocessingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Batch computation too fast (0.0095s.) Setting batch_size=42.
[Parallel(n_jobs=16)]: Done   9 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done  29 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Batch computation too fast (0.0226s.) Setting batch_size=740.
[Parallel(n_jobs=16)]: Done 368 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 914 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Batch computation too fast (0.1559s.) Setting batch_size=1898.
[Parallel(n_jobs=16)]: Done 2856 tasks      | elapsed:    0.2s
[Parallel(n_jobs=16)]: Done 42231 out of 42231 | elapsed:    0.4s finished


|(w',w)| pairs to do calculations for every c = 417,112
|(w',w)| pairs there would be without the k-edit-distance-restriction = 84,125,584
|Cs| = 106295
Est. of total number of calculations to do = 44,336,920,040 = 4.43E+10


If $r$ is `False` and $e$ is `True`, then the dimensions and number of calculations reduce to:

In [262]:
len(Ws)
len(Cs)
num_calcs_case3 = len(Ws) * len(Cs)
"{0:,}".format( num_calcs_case3 )
"{0:.2E}".format( num_calcs_case3 )

9172

106295

'974,937,740'

'9.75E+08'

## Determine dimensions and cells to be calculated vs. ignored

In [263]:
assert (k != 0 and r) or (k != 0 and (not r) and (not e)) or (k == 0 and (not r) and e) or (k > 0 and (not r) and e)

### Option: Only calculate full source wordforms rather than all source prefixes?

In [264]:
print(f"r = {r}")

if r:
    source_sequences = Ps_t
    print("Performing calculations for all source prefixes.")
else:
    source_sequences = Ws_t
    print("Performing calculations only for all complete source wordforms (and *not* all prefixes thereof).")

Ss_t = source_sequences    

print(f"|source_sequences| = {len(source_sequences)}")

r = False
Performing calculations only for all complete source wordforms (and *not* all prefixes thereof).
|source_sequences| = 9172


### Suboption: Given $\neg r$ and a full source wordform $w^*$, only calculate posterior probability $p(\widehat{W} = w^* | W = w^*, C = c)$, $\forall w^*$?

In [265]:
print(f"¬r ∧ e = {not r} ∧ {e} = {(not r) and e}")

if (not r) and e:
    print("Performing calculations only for those cases where the reconstructed wordform exactly matches the source wordform.")
    
else:
    print("Calculations not restricted only to exact reconstructed wordform matches.")

¬r ∧ e = True ∧ True = True
Performing calculations only for those cases where the reconstructed wordform exactly matches the source wordform.


### Suboption: Restrict calculations only to (source sequence, reconstruction sequence) pairs within edit distance $k$?

In [266]:
print(f"k = {k}")

if r or ((not r) and (not e)):
    print(f"Restricting calculations only to (source sequence, reconstructed sequence) pairs within edit distance {k}.")
elif (not r) and e:
    print(f"¬r ∧ e is equivalent to restricting calculations only to (source sequence, reconstructed sequence) pairs within edit distance 0.")
else:
    print("No restrictions on (source sequence, reconstructed sequence) pairs to be calculated.")

k = 2
¬r ∧ e is equivalent to restricting calculations only to (source sequence, reconstructed sequence) pairs within edit distance 0.


### Altogether...

In [267]:
o

'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts'

In [268]:
if r and (0 < k <= 4):
    case = 1    
    myShape = (len(Ws), len(Ps), len(Cs))
    my_num_calcs = num_calcs_case1
    
    my_type = f'pW_RC_k{k}'
    status = '_unnormalized'

if (not r) and (0 < k <= 4) and (not e):
    case = 2
    myShape = (len(Ws), len(Ws), len(Cs))
    my_num_calcs = num_calcs_case2
    
    my_type = f'pW_WC_k{k}'
    status = '_unnormalized'
    
if (not r) and e:
    case = 3
#     myShape = (len(Ws), len(Ws), len(Cs))
    myShape = (len(Ws), len(Cs))
    my_num_calcs = num_calcs_case3

    my_type = f'pW_WC_e'
    status = ''

print(f'Case:\n\t{case}')
print(f'Shape of matrix:\n\t{myShape}')
print('Num elements:')
print("\t{0:,} = {0:.2E}".format(prod(myShape)))
print('Num calculations:')
print("\t{0:,} = {0:.2E}".format(my_num_calcs))
print('Est. size in GB of result:')
print("\t{0:,} = {0:.2E}".format(my_num_calcs * 8 / 1e9))
print("Density:")
print("\t{0:,}".format(my_num_calcs / prod(myShape)))    

print('Result suffix:')
my_fn_suffix = my_type + status
print(f"\t{my_fn_suffix}")

Case:
	3
Shape of matrix:
	(9172, 106295)
Num elements:
	974,937,740 = 9.75E+08
Num calculations:
	974,937,740 = 9.75E+08
Est. size in GB of result:
	7.79950192 = 7.80E+00
Density:
	1.0
Result suffix:
	pW_WC_e


In [269]:
abort_threshold_GB = 250
size_est_GB = (my_num_calcs * 8 / 1e9)
assert size_est_GB <= abort_threshold_GB, "Estimated size of resulting matrix is ≥ {1} GB:\n\tEst. size = {0:,}GB = {0:.2E}GB\n\tAborting calculation.".format(size_est_GB, abort_threshold_GB)

In [ ]:
# if exists(o + my_fn_suffix):
#     unnormalized_posterior = np.load(o + my_fn_suffix + '.npy',
#                                      allow_pickle = False)
# assert not exists(o + my_fn_suffix + '.npy'), 'Unnormalized distribution already found at filepath \n\t{0}'.format( o + my_fn_suffix + '.npy' )

In [ ]:
# unnormalized_posterior_torch = np.zeros(myShape)

## Case 1: $r \land (0 < k \leq 4)$

## Case 2: $\neg r \land (0 < k \leq 4) \land \neg e$

## Case 3: $\neg r \land e$

In [270]:
myShape
blockShape = (myShape[0], 1) #every context is a slice
blockShape

(9172, 106295)

(9172, 1)

In [325]:
dom = tiledb.Domain(tiledb.Dim(name="segWord", domain=(0, myShape[0]-1), tile=blockShape[0], dtype=np.uint32),
                    tiledb.Dim(name="context", domain=(0, myShape[1]-1), tile=blockShape[1], dtype=np.uint32))

schema = tiledb.ArraySchema(domain=dom, sparse=False,
                            attrs=[tiledb.Attr(name="O", dtype=np.float64)])

In [326]:
o
my_fn_suffix
o + '.' + my_fn_suffix

'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts'

'pW_WC_e'

'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_WC_e'

In [329]:
array_name = o + '.' + my_fn_suffix
tiledb.SparseArray.create(array_name, schema)

In [328]:
# rm -r 'LD_Fisher_vocab_in_swbd2003_contexts/LTR_newdic_destressed_aligned_CM_filtered_LM_filtered_in_swbd2003_contexts.pW_WC_e'

In [331]:
# will also work if if given a range of indices and a range of slices...
def slice_writer(slice_context_index, context_slice, A):
# #     with tiledb.SparseArray(arr_name, mode='w') as A:
        A[:,slice_context_index] = context_slice

In [332]:
config = tiledb.Config()
# config["sm.consolidation.steps"] = 10
config["sm.consolidation.steps"] = 600
config["sm.consolidation.step_min_frags"] = 2
config["sm.consolidation.step_max_frags"] = 20
# tiledb.consolidate(array_name, config)

In [ ]:
if cStart != '' and cEnd != '':
    print(f'Context block start and end given: ({cStart}, {cEnd})')
    blockRange = np.arange(cStart, cEnd)
    assert 0 <= cStart <= len(Cs_t), f'0<= cStart <= len(Cs_t) does not hold: {cStart} vs. {len(Cs_t)}'
    assert 0 <= cEnd <= len(Cs_t), f'0<= cEnd <= len(Cs_t) does not hold: {cEnd} vs. {len(Cs_t)}'
else:
    print(f'Calculating all indices from 0 to len(Cs_t) = {len(Cs_t)}')
    blockRange = np.arange(len(Cs_t))


for c_idx in tqdm(blockRange):
    context_slice = p_wstar_wstar_pxtn_by_c(c_idx, m=50, asType='ndarray', parallel=True)

    with tiledb.SparseArray(array_name, mode='w') as A:
        slice_writer(c_idx, context_slice, A)
        
    if c_idx % 100 == 0:
        tiledb.consolidate(config, uri=array_name)